In [1]:
import sys
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.special import expit
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
from copy import deepcopy
from typing import List, Tuple, Dict, Callable


from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

from sklearn import svm
from sklearn import metrics
from copy import deepcopy
import Cython
import random
from pprint import pprint
import time
from joblib import Parallel, delayed, parallel_backend, Memory
import math
from tqdm import tqdm
from sklearn.model_selection import KFold
#import KFold
from statistics import mean
#from apriori_loss import *
#from scikit.sklearn.tree import DecisionTreeClassifierLoss
#from tree_update_loss.tree import *

In [2]:
# Librairies
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline



#Differents functions
# spliting datas

def train_test_split_(data, test_size):
    
    #check if test_size is a float
    if isinstance(test_size, float):
        test_size = round(test_size * len(data))
    
    #retrieve all indices
    indices = data.index.tolist()
    
    #select random indices according to test_size
    test_indices = random.sample(population=indices, k=test_size)
    
    #spliting datas in training & testing
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    
    return train_data, test_data


#Check data purity
def check_purity(data):
    #print("check purity : ", len(np.unique(data[:,-1])) == 1)
    #Return purity value regarding to the number of unique classes
    return  (False,True)[len(np.unique(data[:,-1])) == 1]
    
#Normal classification using majority
def classify(data):
    #get all classes & count occurences
    label = data[:,-1]
    uniques_classes, counts = np.unique(data[:,-1], return_counts = True)
    
    #get index of most common class
    index = counts.argmax()
    
    #return the most common class
    return uniques_classes[index]

# Modified Classification using proposed approach
def modified_classify(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = idx_montant #13
    index_taux = idx_taux #2
    index_nbe = idx_nbe #4
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        else:
            M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
        #print("({} , {})".format(data[line,index_montant], data[line, index_taux]))
    
    #print("({}, {})".format(P, M))
    
    return (0, 1)[M > P]
        

#Calculate all potential splits
def get_splits(data, index_feature_not_selected, col_montant):
    
    #initialize our dict of potential splits
    splits = {}
    
    #initialize all attribute potential splits list
    _, n_cols = data.shape
    #for col in range(n_cols - 1):
    #    if col in index_feature_not_selected or col == col_montant:
    #        continue
    #    splits[col] = list()
    
    # Compute all attributes potential splits
    for col in range(n_cols - 1):
        #get unique datas
        #print("Current col: {}".format(col))
        if col in index_feature_not_selected or col == col_montant:
            continue

        values = np.unique(data[:, col])
        #print("values :", values)
        #populate our dict
        feature_type = FEATURE_TYPES[col]
        if feature_type == "Continous":
            for index in range(1, len(values)):
                if not col in splits:
                    splits[col] = list()
                current_value = values[index]
                #print(current_value)
                previous_value = values[index - 1]
                potential_split = np.mean([current_value, previous_value])
                splits[col].append(potential_split)
        else:
            splits[col] = values
    #print("splits : ", splits)
    return splits

#Spliting data
def split_data(data, feature_col, value):
    
    feature_type = FEATURE_TYPES[feature_col]
    
    #define all masks
    
    if feature_type == "Continous":
        mask_inf = data[:, feature_col] <= value
        mask_sup = data[:, feature_col] > value
    else:
        mask_inf = data[:, feature_col] == value
        mask_sup = data[:, feature_col] != value
    
    #data spliting
    if mask_inf.size == 0:
        data_inf = np.array([])
    else:
        data_inf = data[mask_inf]
        
    if mask_sup.size == 0:
        data_sup = np.array([])
    else:
        data_sup = data[mask_sup]

    return data_inf, data_sup


#Calculate chosen metric
def calculate_metric(data, idx_montant, idx_taux, idx_nbe):
     #get classes
    label_class = data[:, -1]
    #get counts for each class
    _, counts = np.unique(label_class, return_counts=True)
    
    probabilities = counts / counts.sum()
    
    #computing metric value depending on the user choice
    if METRIC == "entropy":
        probabilities = counts / counts.sum()
        computed_metric = sum(probabilities * -np.log2(probabilities))
    elif METRIC == "gini":
        probabilities **=2
        computed_metric = 1- sum(probabilities)
    elif METRIC == "cs":
        #print("cs")
        P  = 0
        M = 0
        n_row , _ = data.shape
        index_montant = idx_montant #13
        index_taux = idx_taux #12
        index_nbe = idx_nbe #4
        for line in range(n_row):
            if data[line,-1] == 0:
                P += data[line,index_montant]
            else:
                M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
        prod = np.prod(probabilities)
        computed_metric = P*prod + M*prod
    
    return computed_metric

# Overall metric value
def overall_metric(data_inf, data_sup, idx_montant, idx_taux, idx_nbe):
    data_all_lenght = len(data_inf) + len(data_sup)
    
    #compute overall metric value
    
    if len(data_inf) == 0:
        metric_data_inf = 0
    else:
        metric_data_inf = (len(data_inf) / data_all_lenght) * calculate_metric(data_inf, idx_montant, idx_taux, idx_nbe)
    
    if len(data_sup) == 0:
        metric_data_sup = 0
    else:
        metric_data_sup = (len(data_sup) / data_all_lenght) * calculate_metric(data_sup, idx_montant, idx_taux, idx_nbe)

    overall_metric= metric_data_inf + metric_data_sup
    return overall_metric

#Modified metric: Here the aim is to compute the total lost that we'll have
def modified_metric(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = idx_montant #13
    index_taux = idx_taux #12
    index_nbe = idx_nbe #4
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        #else:
            #M += (data[line, index_taux]/100) * data[line,index_montant] 
    #print(P)
    return P

#Compute our overall modified metric
def overall_modified_metric(data_inf, data_sup, idx_montant, idx_taux, idx_nbe):
    #get number of datas
    data_all_lenght = len(data_inf) + len(data_sup)
    
    #compute overall metric value
    metric_data_inf = (len(data_inf) / data_all_lenght)*modified_metric(data_inf, idx_montant, idx_taux, idx_nbe)
    metric_data_sup = (len(data_sup) / data_all_lenght)*modified_metric(data_sup, idx_montant, idx_taux, idx_nbe)
    overall_metric= metric_data_inf + metric_data_sup
    #print("({}, {})".format(metric_data_inf, metric_data_sup))
    
    return overall_metric

#parralelized function
def computing_best_column(data, colum_index, value, idx_montant, idx_taux, idx_nbe):
    
    global best_split_column, best_split_value, overall_metric_value
    
    #print("current column: {}".format(colum_index))
    data_inf, data_sup = split_data(data, colum_index, value)
    current_overall = overall_modified_metric(data_inf, data_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
    if current_overall <= overall_metric_value:
        ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
        overall_metric_value = current_overall
        best_split_column = colum_index
        best_split_value = value
        #print("Done !! bests: ({}, {}, {})".format(best_split_column, best_split_value, overall_metric_value))

#determine best split attribute and value
def determine_best_split(data, potential_splits, idx_montant, idx_taux, idx_nbe):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    #global best_split_column, best_split_value, overall_metric_value
    
    #overall_metric_value = 300000000000000
    overall_metric_value = float('inf')
    #with Parallel(n_jobs=-1, backend="threading", require="sharedmem", verbose=5) as parallel:
      #  parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    best_split_columns = 0
    best_split_value = 0
    for colum_index in potential_splits:
        #print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            #print("current value: {}".format(value))
            data_inf, data_sup = split_data(data, colum_index, value)
            current_overall = overall_metric(data_inf, data_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
            if current_overall <= overall_metric_value:
                ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                overall_metric_value = current_overall
                best_split_columns = colum_index
                best_split_value = value
    #print(best_split_value)
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    #print("Final Done !! bests: ({}, {}, {})".format(best_split_columns, best_split_value, overall_metric_value))
    return best_split_columns, best_split_value, overall_metric_value

#building decision Tree
def decision_tree(df, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter=0, min_samples=5, max_depth=5, metric="entropy"):
    
    if counter == 0:
        if isinstance(df, pd.DataFrame):
            data = df.values
        else: 
            data = df
        global COLUMNS_NAMES, FEATURE_TYPES, METRIC
        COLUMNS_NAMES = df.columns[:-1]
        FEATURE_TYPES = determine_feature_types(df)
        METRIC = metric
    else:
        data = df
        
    #base case
    potential_splits = get_splits(data, index_feature_not_selected, idx_montant)
    #print(potential_splits)
    if check_purity(data) or (len(data) < min_samples) or (counter == max_depth) or potential_splits == {}:
        classification = modified_classify(data, idx_montant, idx_taux, idx_nbe)
        return classification
    
    else:
        counter +=1
        #computations for right and left part 
        best_split_column, best_split_value, overall_metric_value = determine_best_split(data, potential_splits, idx_montant, idx_taux, idx_nbe)
        
        #We must change data_inf & data_sup order later
        data_inf, data_sup = split_data(data, best_split_column, best_split_value)
        
        # Creating subTree
        feature_type = FEATURE_TYPES[best_split_column]    
        if feature_type == "Continous":
            question = "{} <= {}".format(COLUMNS_NAMES[best_split_column], best_split_value)
        else:
            question = "{} == {}".format(COLUMNS_NAMES[best_split_column], best_split_value)
        #Adding labels
        labels = " " + str(len(data_inf) + len(data_sup)) + " " + str(overall_metric_value)
        question += labels
        sub_tree = {question: []}
        ###print(question)            
        #index_feature_not_selected.append(best_split_column)
        #left and right
        yes_answer = decision_tree(data_inf, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC)
        no_answer = decision_tree(data_sup, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC)
        
        if yes_answer == no_answer:
            ###print(yes_answer)
            sub_tree = yes_answer
        else:
            #Append left and right part
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree
        
#classify a sample
def classify_sample(sample, tree):
    
    #get node elements
    if not isinstance(tree, dict):
        return tree
    key = list(tree.keys())[0]
    #print(key.split())
    feature, comp_op, value, _, _ = key.split()
    
    if comp_op == "<=":
        if sample[feature] <= float(value):
            answer = tree[key][0]
        else:
            answer = tree[key][1]
    else:
        if str(sample[feature]) == value:
            answer = tree[key][0]
        else:
            answer = tree[key][1]
        
    #test base case
    if not isinstance(answer, dict):
        #print('yes')
        return answer
    else:
        return classify_sample(sample, answer)

#compute accuracy
def my_accuracy(df, tree, classe):
    
    df["classification"] = df.apply(classify_sample, axis=1, args=(tree,))
    df["classification_correct"] = df.classification == df[classe]
    
    accuracy = df.classification_correct.mean()
    
    return accuracy, df
    

# In order to handle non-continous values
# We have to identify all features type in our dataset
def determine_feature_types(data):
    
    features_type  = []
    threshold = 931
    for col in data.columns:
        
        uniques_val = data[col].unique()
        sample = uniques_val[0]
        
        if (isinstance(sample, str)):
            features_type.append("Categorical")
        else:
            features_type.append("Continous")
    
    return features_type


def compute_metrics(df, name_classe, name_montant, name_taux, name_duree):    
    # 1- Accuracy
    accuracy = df.classification_correct.mean()
    
    #2- Recall, precision
    TP = len(df[(df[name_classe] == 1) & (df.classification == 1)])
    FP = len(df[(df[name_classe] == 0) & (df.classification == 1)])
    FN = len(df[(df[name_classe] == 1) & (df.classification == 0)])
    TN = len(df[(df[name_classe] == 0) & (df.classification == 0)])
    
    if TP == 0 and FP == 0:
        Precision = 0.0
    else:
        Precision = TP / (TP + FP)
    if TP == 0 and FN == 0:
        Recall = 0.0
    else:
        Recall = TP / (TP + FN)
    Precision, Recall
    
    #3- Lost, no_win
    #Computing new matrix confusion values
    P = df[name_montant][(df[name_classe] == 0) & (df.classification == 1)].sum()

    df_no_win = df[[name_montant, name_taux, name_duree]][(df[name_classe] == 1) & (df.classification == 0)]
    df_no_win[name_taux] /=100
    df_no_win["NoWin"] = df_no_win[name_montant] * df_no_win[name_taux] * df_no_win[name_duree]/12

    M = df_no_win.NoWin.sum()
    P, M
    computed_metric = P + M
    # Error
    #error = (FP / (TP + FP)) + (FN / FN + TN)
    
    
    #label_class = list(df[name_classe])
    #get counts for each class
    #_, counts = np.unique(label_class, return_counts=True)
    
    #probabilities = counts / counts.sum()
    
    #prod = np.prod(probabilities)
    #computed_metric = P*prod + M*prod
        
    #Friedman Test
    #f, _ = friedmanchisquare(df.kredit.values, df.classification.values)
    
    #resulting metric list
    result = [(accuracy, Precision, Recall), (P, M), (TP, FP, FN, TN)]
    
    return result, computed_metric

def inverse(x):
    if x == 0:
        return 1
    else:
        return 0

def compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree):    
    # 1- Accuracy
    accuracy = df.classification_correct.mean()
    
    #2- Recall, precision
    set_tp = df[(df[name_classe] == 1) & (df.classification == 1)]
    set_fp = df[(df[name_classe] == 0) & (df.classification == 1)]
    set_fn = df[(df[name_classe] == 1) & (df.classification == 0)]
    set_tn = df[(df[name_classe] == 0) & (df.classification == 0)]
    
    
    TP = len(set_tp)
    FP = len(set_fp)
    FN = len(set_fn)
    TN = len(set_tn)
    
    #print("TP : ", TP)
    #print("FP : ", FP)
    #print("FN : ", FN)
    #print("TN : ", TN)
    
    cout_tp = sum(set_tp[name_montant] * (set_tp[name_taux] / 100) * set_tp[name_duree] / 12)
    cout_tn = sum(set_tn[name_montant] * (1-alpha))
    cout_fp = sum(set_fp[name_montant] * (1-alpha))#-
    cout_fn = sum(set_fn[name_montant] * (set_fn[name_taux] / 100) * set_fn[name_duree] / 12)#-
    
    
    cout = 0.0
    if type_cout == "total":
        cout_to = cout_tp - cout_fp - cout_fn + cout_tn
    elif type_cout == "optimiste":
        cout_op = cout_tp + cout_tn
    elif type_cout == "pessimiste":
        cout_pe = cout_fn + cout_fp
    elif type_cout == "reel":
        cout_re = cout_tp + cout_fp
    
    #df_v = df.copy()
    #df_v["inverse"] = df_v[name_classe].apply(inverse)
    #set_fp = df_v[(df_v[name_classe] == 0) & (df_v['inverse'] == 1)]
    #set_fn = df_v[(df_v[name_classe] == 1) & (df_v['inverse'] == 0)]
    #cout_fp = sum(set_fp[name_montant] * (1-alpha))#-
    #cout_fn = sum(set_fn[name_montant] * (set_fn[name_taux] / 100) * set_fn[name_duree] / 12)
    #s_all = cout_fn + cout_fp
    
    
    #cout = (cout_tp + cout_tn)/(cout_tp + cout_tn - cout_fp - cout_fn)
    #cout = cout_pe/s_all
    cout = cout_pe
    if TP == 0 and FP == 0:
        Precision = 0.0
    else:
        Precision = TP / (TP + FP)
    if TP == 0 and FN == 0:
        Recall = 0.0
    else:
        Recall = TP / (TP + FN)
        
    #resulting metric list
    result = (cout_pe, accuracy, Precision, Recall)
    
    return result, cout





def extract_feature_select_tree(my_tree):
    j = 0
    stock_tree = []
    list_key = []
    stock_tree.append(my_tree)
    while j <= len(stock_tree)-1:
        #print(stock_tree)
        tree = stock_tree[j]
        if type(tree) is dict:
            for key, value in tree.items():
                list_key.append(key)
                if type(value[0]) is dict:
                    stock_tree.append(value[0])
                if type(value[1]) is dict:
                    stock_tree.append(value[1])
        j += 1   
    for i in range(len(list_key)):
        list_key[i] = list_key[i].split(" ")[0]
    list_key = list(set(list_key))

    return list_key


#determine_feature_types -- ok
#modified_classify -- not ok
#get_splits -- ok
#determine_best_split 
#split_data -- ok
#overall_metric

In [3]:

def check_purity_1(data):
    #print("check purity : ", len(np.unique(data[:,-1])) == 1)
    #Return purity value regarding to the number of unique classes
    return  (False,True)[len(np.unique(data[:,-1])) == 1]
    
#Normal classification using majority
def classify_1(data, index_montant):
    #get all classes & count occurences
    M = sum(data[data[:,-1] == 1, index_montant])
    P = sum(data[data[:,-1] == 0, index_montant])
    #get index of most common class
    #return the most common class
    return (0, 1)[M > P]

def modified_classify_update_1(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = idx_montant #13
    index_taux = idx_taux #2
    index_nbe = idx_nbe #4
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        else:
            M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
        #print("({} , {})".format(data[line,index_montant], data[line, index_taux]))
    
    #print("({}, {})".format(P, M))
    
    return (0, 1)[M > P]


# Modified Classification using proposed approach
def modified_classify_1(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    M = sum(data[data[:,-1] == 1, idx_montant])
    P = sum(data[data[:,-1] == 0, idx_montant])
    return (0, 1)[M > P]


#Calculate all potential splits
def get_splits_1(data, index_feature_not_selected, col_montant):
    
    #initialize our dict of potential splits
    splits = {}
    
    #initialize all attribute potential splits list
    _, n_cols = data.shape
    #for col in range(n_cols - 1):
    #    if col in index_feature_not_selected or col == col_montant:
    #        continue
    #    splits[col] = list()
    
    # Compute all attributes potential splits
    for col in range(n_cols - 1):
        #get unique datas
        #print("Current col: {}".format(col))
        if col in index_feature_not_selected or col == col_montant:
            continue

        values = np.unique(data[:, col])
        #print("values :", values)
        #populate our dict
        feature_type = FEATURE_TYPES_1[col]
        if feature_type == "Continous":
            for index in range(1, len(values)):
                if not col in splits:
                    splits[col] = list()
                current_value = values[index]
                #print(current_value)
                previous_value = values[index - 1]
                potential_split = np.mean([current_value, previous_value])
                splits[col].append(potential_split)
        else:
            splits[col] = values
    #print("splits : ", splits)
    return splits

#Spliting data
def split_data_1(data, feature_col, value):
    
    feature_type = FEATURE_TYPES_1[feature_col]
    
    #define all masks
    
    if feature_type == "Continous":
        mask_inf = data[:, feature_col] <= value
        mask_sup = data[:, feature_col] > value
    else:
        mask_inf = data[:, feature_col] == value
        mask_sup = data[:, feature_col] != value
    
    #data spliting
    if mask_inf.size == 0:
        data_inf = np.array([])
    else:
        data_inf = data[mask_inf]
        
    if mask_sup.size == 0:
        data_sup = np.array([])
    else:
        data_sup = data[mask_sup]
    
    return data_inf, data_sup


#Calculate chosen metric
def calculate_metric_1(data, idx_montant, idx_taux, idx_nbe):
     #get classes
    label_class = data[:, -1]
    #get counts for each class
    label, counts = np.unique(label_class, return_counts=True)
    computed_metric = 0.0
    if len(label) != 0:
        n_row , _ = data.shape
        counts_loss = np.zeros(len(label))
        for i in range(len(label)):
            data_label = data[data[:, -1] == label[i],:]
            counts_loss[i] = sum(data_label[:, idx_montant])
        #sum_counts_loss = sum(counts_loss)
        #for i in range(len(counts_loss)):
        #    counts_loss[i] /= sum_counts_loss 
        #probabilities_loss = counts_loss
        probabilities_loss = counts_loss / counts_loss.sum()
        #computing metric value depending on the user choice
        if METRIC_1 == "entropy":
            #probabilities = counts / counts.sum()
            computed_metric = sum(probabilities_loss * -np.log2(probabilities_loss))
        elif METRIC_1 == "gini":
            probabilities_loss **=2
            computed_metric = 1 - sum(probabilities_loss)
        elif METRIC_1 == "cs":
            #print("cs")
            P  = 0
            M = 0
            n_row , _ = data.shape
            index_montant = idx_montant #13
            index_taux = idx_taux #12
            index_nbe = idx_nbe #4
            for line in range(n_row):
                if data[line,-1] == 0:
                    P += data[line,index_montant]
                else:
                    M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
            prod = np.prod(probabilities_loss)
            computed_metric = P*prod + M*prod
    
    return computed_metric



def calculate_metric_update_1(data, idx_montant, idx_taux, idx_nbe):
    
    label_class = data[:, -1]
    #get counts for each class
    label, counts = np.unique(label_class, return_counts=True)
    computed_metric = 0.0
    if len(label) != 0:    
        P  = 0
        M = 0
        n_row , _ = data.shape
        index_montant = idx_montant #13
        index_taux = idx_taux #12
        index_nbe = idx_nbe #4
        label, _ = np.unique(data[:, - 1], return_counts=True)
        counts_loss = np.zeros(2)
        for line in range(n_row):
            if data[line,-1] == 0:
                P += data[line,index_montant]
            else:
                M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
        
        for i in label:
            if int(i) == 0:
                counts_loss[int(i)] = P
            else:
                counts_loss[int(i)] = M
        probabilities_loss = counts_loss / counts_loss.sum()
        #computing metric value depending on the user choice
        if METRIC_1 == "entropy":
            #probabilities = counts / counts.sum()
            computed_metric = sum(probabilities_loss * -np.log2(probabilities_loss))
        elif METRIC_1 == "gini":
            probabilities_loss **=2
            computed_metric = 1 - sum(probabilities_loss)    
    return computed_metric

# Overall metric value
def overall_metric_1(data_inf, data_sup, idx_montant, idx_taux, idx_nbe):
    inf_loss = sum(data_inf[:, idx_montant])
    sup_loss = sum(data_sup[:, idx_montant])
    
    data_all_lenght_loss = inf_loss + sup_loss
    
    #compute overall metric value
    if inf_loss == 0:
        metric_data_inf = 0
    else:
        metric_data_inf = (inf_loss / data_all_lenght_loss) * calculate_metric_update_1(data_inf, idx_montant, idx_taux, idx_nbe)
    
    if sup_loss == 0:
        metric_data_sup = 0
    else:
        metric_data_sup = (sup_loss / data_all_lenght_loss) * calculate_metric_update_1(data_sup, idx_montant, idx_taux, idx_nbe)
        
    overall_metric= metric_data_inf + metric_data_sup
    
    return overall_metric


#determine best split attribute and value
def determine_best_split_1(data, potential_splits, idx_montant, idx_taux, idx_nbe):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    #global best_split_column, best_split_value, overall_metric_value
    
    #overall_metric_value = 300000000000000
    overall_metric_value = float('inf')
    #with Parallel(n_jobs=-1, backend="threading", require="sharedmem", verbose=5) as parallel:
      #  parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    best_split_columns = 0
    best_split_value = 0
    for colum_index in potential_splits:
        #print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            #print("current value: {}".format(value))
            data_inf, data_sup = split_data_1(data, colum_index, value)
            current_overall = overall_metric_1(data_inf, data_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
            if current_overall <= overall_metric_value:
                ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                overall_metric_value = current_overall
                best_split_columns = colum_index
                best_split_value = value
    #print(best_split_value)
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    #print("Final Done !! bests: ({}, {}, {})".format(best_split_columns, best_split_value, overall_metric_value))
    return best_split_columns, best_split_value, overall_metric_value

#building decision Tree
def decision_tree_1(df, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter=0, min_samples=5, max_depth=5, metric="entropy"):
    
    if counter == 0:
        if isinstance(df, pd.DataFrame):
            data = df.values
        else: 
            data = df
        global COLUMNS_NAMES_1, FEATURE_TYPES_1, METRIC_1
        COLUMNS_NAMES_1 = df.columns[:-1]
        FEATURE_TYPES_1 = determine_feature_types(df)
        METRIC_1 = metric
    else:
        data = df
        
    #base case
    potential_splits = get_splits_1(data, index_feature_not_selected, idx_montant)
    #print(potential_splits)
    if check_purity_1(data) or (len(data) < min_samples) or (counter == max_depth) or potential_splits == {}:
        classification = modified_classify_update_1(data, idx_montant, idx_taux, idx_nbe)
        return classification
    
    else:
        counter +=1
        #computations for right and left part 
        best_split_column, best_split_value, overall_metric_value = determine_best_split_1(data, potential_splits, idx_montant, idx_taux, idx_nbe)
        
        #We must change data_inf & data_sup order later
        data_inf, data_sup = split_data_1(data, best_split_column, best_split_value)
        
        # Creating subTree
        feature_type = FEATURE_TYPES_1[best_split_column]    
        if feature_type == "Continous":
            question = "{} <= {}".format(COLUMNS_NAMES_1[best_split_column], best_split_value)
        else:
            question = "{} == {}".format(COLUMNS_NAMES_1[best_split_column], best_split_value)
        #Adding labels
        labels = " " + str(len(data_inf) + len(data_sup)) + " " + str(overall_metric_value)
        question += labels
        sub_tree = {question: []}
        ###print(question)            
        
        #left and right
        yes_answer = decision_tree_1(data_inf, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC_1)
        no_answer = decision_tree_1(data_sup, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC_1)
        
        if yes_answer == no_answer:
            ###print(yes_answer)
            sub_tree = yes_answer
        else:
            #Append left and right part
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [4]:

def check_purity_2(data):
    #print("check purity : ", len(np.unique(data[:,-1])) == 1)
    #Return purity value regarding to the number of unique classes
    return  (False,True)[len(np.unique(data[:,-1])) == 1]
    
#Normal classification using majority
def classify_2(data):
    #get all classes & count occurences
    label = data[:,-1]
    uniques_classes, counts = np.unique(data[:,-1], return_counts = True)
    
    #get index of most common class
    index = counts.argmax()
    
    #return the most common class
    return uniques_classes[index]

# Modified Classification using proposed approach
def modified_classify_2(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = idx_montant #13
    index_taux = idx_taux #2
    index_nbe = idx_nbe #4
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        else:
            M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
        #print("({} , {})".format(data[line,index_montant], data[line, index_taux]))
    
    return (0, 1)[M > P]       

#Calculate all potential splits
def get_splits_2(data, index_feature_not_selected, col_montant):
    
    #initialize our dict of potential splits
    splits = {}
    
    #initialize all attribute potential splits list
    _, n_cols = data.shape
    #for col in range(n_cols - 1):
    #    if col in index_feature_not_selected or col == col_montant:
    #        continue
    #    splits[col] = list()
    
    # Compute all attributes potential splits
    for col in range(n_cols - 1):
        #get unique datas
        #print("Current col: {}".format(col))
        if col in index_feature_not_selected or col == col_montant:
            continue

        values = np.unique(data[:, col])
        #print("values :", values)
        #populate our dict
        feature_type = FEATURE_TYPES_2[col]
        if feature_type == "Continous":
            for index in range(1, len(values)):
                if not col in splits:
                    splits[col] = list()
                current_value = values[index]
                #print(current_value)
                previous_value = values[index - 1]
                potential_split = np.mean([current_value, previous_value])
                splits[col].append(potential_split)
        else:
            splits[col] = values
    #print("splits : ", splits)
    return splits

#Spliting data
def split_data_2(data, feature_col, value):
    
    feature_type = FEATURE_TYPES_2[feature_col]
    
    #define all masks
    
    if feature_type == "Continous":
        mask_inf = data[:, feature_col] <= value
        mask_sup = data[:, feature_col] > value
    else:
        mask_inf = data[:, feature_col] == value
        mask_sup = data[:, feature_col] != value
    
    #data spliting
    if mask_inf.size == 0:
        data_inf = np.array([])
    else:
        data_inf = data[mask_inf]
        
    if mask_sup.size == 0:
        data_sup = np.array([])
    else:
        data_sup = data[mask_sup]
    
    return data_inf, data_sup


#Calculate chosen metric
def calculate_metric_2(matrix_conf):
    if METRIC_2 == "total":
        computed_metric = matrix_conf[0][0] - matrix_conf[1][0] - matrix_conf[0][1] + matrix_conf[1][1]
    elif METRIC_2 == "pessimiste":
        computed_metric = matrix_conf[1][0] + matrix_conf[0][1]
    elif METRIC_2 == "optimiste":
        computed_metric = matrix_conf[0][0] + matrix_conf[1][1]
    elif METRIC_2 == "reel":
        computed_metric = matrix_conf[0][0] - matrix_conf[1][0]
    
    return computed_metric

# Overall metric value
def overall_metric_2(data_inf, data_sup, classe_maj_inf, classe_maj_sup, idx_montant, idx_taux, idx_nbe):
    matrix_conf = np.zeros((2,2))
    data_inf_0 = data_inf[data_inf[:, -1] == 0,:]
    data_inf_1 = data_inf[data_inf[:, -1] == 1,:]
    
    data_sup_0 = data_sup[data_sup[:, -1] == 0,:]
    data_sup_1 = data_sup[data_sup[:, -1] == 1,:]
    
    
    if classe_maj_inf == 0:
        matrix_conf[0][classe_maj_inf] = sum(data_inf_0[:, idx_montant]) * (1.0 - ALPHA_2)
        matrix_conf[1][classe_maj_inf] = sum(data_inf_1[:, idx_montant] * (data_inf_1[:, idx_taux] / 100) * (data_inf_1[:, idx_nbe] / 12))# -
        matrix_conf[0][classe_maj_sup] = sum(data_sup_0[:, idx_montant]) * (1.0 - ALPHA_2)# -
        matrix_conf[1][classe_maj_sup] = sum(data_sup_1[:, idx_montant] * (data_sup_1[:, idx_taux] / 100) * (data_sup_1[:, idx_nbe] / 12))
    else:
        matrix_conf[0][classe_maj_inf] = sum(data_inf_0[:, idx_montant]) * (1.0 - ALPHA_2)# -
        matrix_conf[1][classe_maj_inf] = sum(data_inf_1[:, idx_montant] * (data_inf_1[:, idx_taux] / 100) * (data_inf_1[:, idx_nbe] / 12))
        matrix_conf[0][classe_maj_sup] = sum(data_sup_0[:, idx_montant]) * (1.0 - ALPHA_2)
        matrix_conf[1][classe_maj_sup] = sum(data_sup_1[:, idx_montant] * (data_sup_1[:, idx_taux] / 100) * (data_sup_1[:, idx_nbe] / 12))#-
    
    return calculate_metric_2(matrix_conf)

def overall_metric_update_2(data_inf, data_sup, classe_maj_inf, classe_maj_sup, idx_montant, idx_taux, idx_nbe):
    
    if classe_maj_inf != classe_maj_sup:
        matrix_conf = np.zeros((2,2))
        data_inf_0 = data_inf[data_inf[:, -1] == 0,:]
        data_inf_1 = data_inf[data_inf[:, -1] == 1,:]

        data_sup_0 = data_sup[data_sup[:, -1] == 0,:]
        data_sup_1 = data_sup[data_sup[:, -1] == 1,:]
        
        n_row_inf_0 , _ = data_inf_0.shape
        n_row_inf_1 , _ = data_inf_1.shape
        n_row_sup_0 , _ = data_sup_0.shape
        n_row_sup_1 , _ = data_sup_1.shape


        if classe_maj_inf == 0:
            matrix_conf[0][classe_maj_inf] = sum(data_inf_0[:, idx_montant]) * (1.0 - ALPHA_2)
            matrix_conf[1][classe_maj_inf] = sum(data_inf_1[:, idx_montant] * (data_inf_1[:, idx_taux] / 100) * (data_inf_1[:, idx_nbe] / 12))# -
            matrix_conf[0][classe_maj_sup] = sum(data_sup_0[:, idx_montant]) * (1.0 - ALPHA_2)# -
            matrix_conf[1][classe_maj_sup] = sum(data_sup_1[:, idx_montant] * (data_sup_1[:, idx_taux] / 100) * (data_sup_1[:, idx_nbe] / 12))
        else:
            matrix_conf[0][classe_maj_inf] = sum(data_inf_0[:, idx_montant]) * (1.0 - ALPHA_2)# -
            matrix_conf[1][classe_maj_inf] = sum(data_inf_1[:, idx_montant] * (data_inf_1[:, idx_taux] / 100) * (data_inf_1[:, idx_nbe] / 12))
            matrix_conf[0][classe_maj_sup] = sum(data_sup_0[:, idx_montant]) * (1.0 - ALPHA_2)
            matrix_conf[1][classe_maj_sup] = sum(data_sup_1[:, idx_montant] * (data_sup_1[:, idx_taux] / 100) * (data_sup_1[:, idx_nbe] / 12))#-        
        overall_metric = calculate_metric_2(matrix_conf)
    else:
        optimisation_min = ['pessimiste']
        if METRIC_2 in optimisation_min:
            overall_metric = float('inf')
        else:
            overall_metric = -float('inf')
        """
        global METRIC
        METRIC = "cs"
        data_all_lenght = len(data_inf) + len(data_sup)
    
        #compute overall metric value
        metric_data_inf = (len(data_inf) / data_all_lenght) * calculate_metric(data_inf, idx_montant, idx_taux, idx_nbe)
        metric_data_sup = (len(data_sup) / data_all_lenght) * calculate_metric(data_sup, idx_montant, idx_taux, idx_nbe)
        overall_metric = metric_data_inf + metric_data_sup
        """
    return overall_metric




def determine_best_split_update_2(data, potential_splits, idx_montant, idx_taux, idx_nbe):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    #global best_split_column, best_split_value, overall_metric_value
    
    #overall_metric_value = 300000000000000
    optimisation_max = ['total', 'optimiste', 'reel']
    optimisation_min = ['pessimiste']
    if METRIC_2 in optimisation_min:
        overall_metric_value = float('inf')
    else:
        overall_metric_value = -float('inf')
    #with Parallel(n_jobs=-1, backend="threading", require="sharedmem", verbose=5) as parallel:
      #  parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    best_split_columns = 0
    best_split_value = 0
    for colum_index in potential_splits:
        #print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            #print("current value: {}".format(value))
            data_inf, data_sup = split_data_2(data, colum_index, value)
            #print("data_inf[:, -1] : ", data_inf[:, -1])
            #print("data_sup[:, -1] : ", data_sup[:, -1])
            #print(type(data_inf))
            
            if len(data_inf[:, -1]) != 0 and len(data_sup[:, -1]) == 0:
                #print("data_sup[:, -1] : ", data_sup[:, -1])
                bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
                classe_maj_inf = bincount_inf.argmax()
                if classe_maj_inf == 0:
                    classe_maj_sup = 1
                else:
                    classe_maj_sup = 0
            elif len(data_inf[:, -1]) == 0 and len(data_sup[:, -1]) != 0:
                #print("data_inf[:, -1] : ", data_inf[:, -1])
                bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
                classe_maj_sup = bincount_sup.argmax()
                if classe_maj_sup == 0:
                    classe_maj_inf = 1
                else:
                    classe_maj_inf = 0
            else:
                #print("data_inf[:, -1] : ", data_inf[:, -1])
                #print("data_sup[:, -1] : ", data_sup[:, -1])
                bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
                bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
                classe_maj_inf = bincount_inf.argmax()
                classe_maj_sup = bincount_sup.argmax()        
        

            #classe_maj_inf = 0
            #classe_maj_sup = 1
            current_overall = overall_metric_update_2(data_inf, data_sup, classe_maj_inf, classe_maj_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
            
            if METRIC_2 in optimisation_min:
                if current_overall <= overall_metric_value:
                    ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                    overall_metric_value = current_overall
                    best_split_columns = colum_index
                    best_split_value = value
            else:
                if current_overall >= overall_metric_value:
                    ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                    overall_metric_value = current_overall
                    best_split_columns = colum_index
                    best_split_value = value

    #print(best_split_value)
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    #print("Final Done !! bests: ({}, {}, {})".format(best_split_columns, best_split_value, overall_metric_value))
    return best_split_columns, best_split_value, overall_metric_value





def determine_best_split_update_v1_2(data, potential_splits, idx_montant, idx_taux, idx_nbe):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    #global best_split_column, best_split_value, overall_metric_value
    
    #overall_metric_value = 300000000000000
    optimisation_max = ['total', 'optimiste', 'reel']
    optimisation_min = ['pessimiste']
    if METRIC_2 in optimisation_min:
        overall_metric_value = float('inf')
    else:
        overall_metric_value = -float('inf')
    #with Parallel(n_jobs=-1, backend="threading", require="sharedmem", verbose=5) as parallel:
      #  parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    best_split_columns = 0
    best_split_value = 0
    for colum_index in potential_splits:
        #print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            #print("current value: {}".format(value))
            data_inf, data_sup = split_data_2(data, colum_index, value)
            #print("data_inf[:, -1] : ", data_inf[:, -1])
            #print("data_sup[:, -1] : ", data_sup[:, -1])
            #print(type(data_inf))
            
            if len(data_inf[:, -1]) != 0 and len(data_sup[:, -1]) == 0:
                #print("data_sup[:, -1] : ", data_sup[:, -1])
                bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
                classe_maj_inf = bincount_inf.argmax()
                if classe_maj_inf == 0:
                    classe_maj_sup = 1
                else:
                    classe_maj_sup = 0
            elif len(data_inf[:, -1]) == 0 and len(data_sup[:, -1]) != 0:
                #print("data_inf[:, -1] : ", data_inf[:, -1])
                bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
                classe_maj_sup = bincount_sup.argmax()
                if classe_maj_sup == 0:
                    classe_maj_inf = 1
                else:
                    classe_maj_inf = 0
            else:
                #print("data_inf[:, -1] : ", data_inf[:, -1])
                #print("data_sup[:, -1] : ", data_sup[:, -1])
                bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
                bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
                classe_maj_inf = bincount_inf.argmax()
                classe_maj_sup = bincount_sup.argmax()        
        
            
            #if classe_maj_inf == classe_maj_sup:
            classe_maj_inf = modified_classify_2(data_inf, idx_montant, idx_taux, idx_nbe)
            classe_maj_inf = modified_classify_2(data_sup, idx_montant, idx_taux, idx_nbe)    
            #classe_maj_inf = 0
            #classe_maj_sup = 1
            current_overall = overall_metric_update_2(data_inf, data_sup, classe_maj_inf, classe_maj_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
            
            if METRIC_2 in optimisation_min:
                if current_overall <= overall_metric_value:
                    ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                    overall_metric_value = current_overall
                    best_split_columns = colum_index
                    best_split_value = value
            else:
                if current_overall >= overall_metric_value:
                    ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                    overall_metric_value = current_overall
                    best_split_columns = colum_index
                    best_split_value = value

    #print(best_split_value)
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    #print("Final Done !! bests: ({}, {}, {})".format(best_split_columns, best_split_value, overall_metric_value))
    return best_split_columns, best_split_value, overall_metric_value






#determine best split attribute and value
def determine_best_split_2(data, potential_splits, idx_montant, idx_taux, idx_nbe):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    #global best_split_column, best_split_value, overall_metric_value
    
    #overall_metric_value = 300000000000000
    optimisation_max = ['total', 'optimiste', 'reel']
    optimisation_min = ['pessimiste']
    if METRIC_2 in optimisation_min:
        overall_metric_value = float('inf')
    else:
        overall_metric_value = -float('inf')
    #with Parallel(n_jobs=-1, backend="threading", require="sharedmem", verbose=5) as parallel:
      #  parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    best_split_columns = 0
    best_split_value = 0
    for colum_index in potential_splits:
        #print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            #print("current value: {}".format(value))
            data_inf, data_sup = split_data_2(data, colum_index, value)
            #print(np.array(data_inf[:, -1], dtype=np.int64))
            #print(type(data_inf))
            if np.any(data_inf[:, -1]) != False and np.any(data_sup[:, -1]) == False:
                bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
                classe_maj_inf = bincount_inf.argmax()
                if classe_maj_inf == 0:
                    classe_maj_sup = 1
                else:
                    classe_maj_sup = 0
            elif np.any(data_inf[:, -1]) == False and np.any(data_sup[:, -1]) != False:
                bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
                classe_maj_sup = bincount_sup.argmax()
                if classe_maj_sup == 0:
                    classe_maj_inf = 1
                else:
                    classe_maj_inf = 0
            else:
                #print("data_inf[:, -1] : ", data_inf[:, -1])
                #print("data_sup[:, -1] : ", data_sup[:, -1])
                bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
                bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
                classe_maj_inf = bincount_inf.argmax()
                classe_maj_sup = bincount_sup.argmax()        
                
            if classe_maj_inf == classe_maj_sup:
                if max(bincount_inf) > max(bincount_sup):
                    if classe_maj_inf == 1:
                        classe_maj_sup = 0
                    else:
                        classe_maj_sup = 1
                else:
                    if classe_maj_sup == 1:
                        classe_maj_inf = 0
                    else:
                        classe_maj_inf = 1
            #classe_maj_inf = 0
            #classe_maj_sup = 1
            current_overall = overall_metric_2(data_inf, data_sup, classe_maj_inf, classe_maj_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
            
            if METRIC_2 in optimisation_min:
                if current_overall <= overall_metric_value:
                    ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                    overall_metric_value = current_overall
                    best_split_columns = colum_index
                    best_split_value = value
            else:
                if current_overall >= overall_metric_value:
                    ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                    overall_metric_value = current_overall
                    best_split_columns = colum_index
                    best_split_value = value

    #print(best_split_value)
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    #print("Final Done !! bests: ({}, {}, {})".format(best_split_columns, best_split_value, overall_metric_value))
    return best_split_columns, best_split_value, overall_metric_value

#building decision Tree
def decision_tree_2(df, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter=0, min_samples=5, max_depth=5, metric="entropy"):
    
    if counter == 0:
        if isinstance(df, pd.DataFrame):
            data = df.values
        else: 
            data = df
        global COLUMNS_NAMES_2, FEATURE_TYPES_2, METRIC_2
        COLUMNS_NAMES_2 = df.columns[:-1]
        FEATURE_TYPES_2 = determine_feature_types(df)
        METRIC_2 = metric
    else:
        data = df
        
    #base case
    potential_splits = get_splits_2(data, index_feature_not_selected, idx_montant)
    #print(potential_splits)
    if check_purity_2(data) or (len(data) < min_samples) or (counter == max_depth) or potential_splits == {}:
        classification = modified_classify_2(data, idx_montant, idx_taux, idx_nbe)
        return classification
    
    else:
        counter +=1
        #computations for right and left part 
        best_split_column, best_split_value, overall_metric_value = determine_best_split_update_2(data, potential_splits, idx_montant, idx_taux, idx_nbe)
        
        #We must change data_inf & data_sup order later
        data_inf, data_sup = split_data_2(data, best_split_column, best_split_value)
        
        # Creating subTree
        feature_type = FEATURE_TYPES_2[best_split_column]    
        if feature_type == "Continous":
            question = "{} <= {}".format(COLUMNS_NAMES_2[best_split_column], best_split_value)
        else:
            question = "{} == {}".format(COLUMNS_NAMES_2[best_split_column], best_split_value)
        #Adding labels
        labels = " " + str(len(data_inf) + len(data_sup)) + " " + str(overall_metric_value)
        question += labels
        sub_tree = {question: []}
        ###print(question)            
        #index_feature_not_selected.append(best_split_column)
        #left and right
        yes_answer = decision_tree_2(data_inf, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC_2)
        no_answer = decision_tree_2(data_sup, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC_2)
        
        if yes_answer == no_answer:
            ###print(yes_answer)
            sub_tree = yes_answer
        else:
            #Append left and right part
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [5]:
#Check data purity
def check_purity_3(data):
    #print("check purity : ", len(np.unique(data[:,-1])) == 1)
    #Return purity value regarding to the number of unique classes
    return  (False,True)[len(np.unique(data[:,-1])) == 1]
    
#Normal classification using majority
def classify_3(data):
    #get all classes & count occurences
    label = data[:,-1]
    uniques_classes, counts = np.unique(data[:,-1], return_counts = True)
    
    #get index of most common class
    index = counts.argmax()
    
    #return the most common class
    return uniques_classes[index]

# Modified Classification using proposed approach
def modified_classify_3(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = idx_montant #13
    index_taux = idx_taux #2
    index_nbe = idx_nbe #4
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        else:
            M += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
        #print("({} , {})".format(data[line,index_montant], data[line, index_taux]))
    
    #print("({}, {})".format(P, M))
    
    return (0, 1)[M > P]
        

#Calculate all potential splits
def get_splits_3(data, index_feature_not_selected, col_montant):
    
    #initialize our dict of potential splits
    splits = {}
    
    #initialize all attribute potential splits list
    _, n_cols = data.shape
    #for col in range(n_cols - 1):
    #    if col in index_feature_not_selected or col == col_montant:
    #        continue
    #    splits[col] = list()
    
    # Compute all attributes potential splits
    for col in range(n_cols - 1):
        #get unique datas
        #print("Current col: {}".format(col))
        if col in index_feature_not_selected or col == col_montant:
            continue

        values = np.unique(data[:, col])
        #print("values :", values)
        #populate our dict
        feature_type = FEATURE_TYPES[col]
        if feature_type == "Continous":
            for index in range(1, len(values)):
                if not col in splits:
                    splits[col] = list()
                current_value = values[index]
                #print(current_value)
                previous_value = values[index - 1]
                potential_split = np.mean([current_value, previous_value])
                splits[col].append(potential_split)
        else:
            splits[col] = values
    #print("splits : ", splits)
    return splits

#Spliting data
def split_data_3(data, feature_col, value):
    
    feature_type = FEATURE_TYPES[feature_col]
    
    #define all masks
    
    if feature_type == "Continous":
        mask_inf = data[:, feature_col] <= value
        mask_sup = data[:, feature_col] > value
    else:
        mask_inf = data[:, feature_col] == value
        mask_sup = data[:, feature_col] != value
    
    #data spliting
    data_inf = data[mask_inf]
    data_sup = data[mask_sup]
    
    return data_inf, data_sup


#Calculate chosen metric
def calculate_metric_3(data, classe_pred, idx_montant, idx_taux, idx_nbe):
     #get classes
    label_class = data[:, -1]
    #get counts for each class
    _, counts = np.unique(label_class, return_counts=True)
    
    probabilities = counts / counts.sum()
    
    #computing metric value depending on the user choice
    if METRIC == "entropy":
        probabilities = counts / counts.sum()
        computed_metric = sum(probabilities * -np.log2(probabilities))
    elif METRIC == "gini":
        probabilities **=2
        computed_metric = 1- sum(probabilities)
    elif METRIC == "cs":
        #print("cs")
        P  = 0
        M = 0
        n_row , _ = data.shape
        index_montant = idx_montant #13
        index_taux = idx_taux #12
        index_nbe = idx_nbe #4
        i = 0
        for line in range(n_row):
            if data[line,-1] == 0 and classe_pred == 1:
                P += data[line,index_montant]
                i += 1
            elif data[line,-1] == 1 and classe_pred == 0:
                P += (data[line, index_taux]/100) * data[line,index_montant] * data[line,index_nbe]/12
                i += 1
        #computed_metric = (i/counts.sum())*P
        computed_metric = P
    
    return computed_metric

# Overall metric value
def overall_metric_3(data_inf, data_sup, idx_montant, idx_taux, idx_nbe):
    data_all_lenght = len(data_inf) + len(data_sup)
    if len(data_inf[:, -1]) != 0 and len(data_sup[:, -1]) == 0:
        #print("data_sup[:, -1] : ", data_sup[:, -1])
        bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
        classe_maj_inf = bincount_inf.argmax()
        if classe_maj_inf == 0:
            classe_maj_sup = 1
        else:
            classe_maj_sup = 0
    elif len(data_inf[:, -1]) == 0 and len(data_sup[:, -1]) != 0:
        #print("data_inf[:, -1] : ", data_inf[:, -1])
        bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
        classe_maj_sup = bincount_sup.argmax()
        if classe_maj_sup == 0:
            classe_maj_inf = 1
        else:
            classe_maj_inf = 0
    else:
        #print("data_inf[:, -1] : ", data_inf[:, -1])
        #print("data_sup[:, -1] : ", data_sup[:, -1])
        bincount_inf = np.bincount(np.array(data_inf[:, -1], dtype=np.int64))
        bincount_sup = np.bincount(np.array(data_sup[:, -1], dtype=np.int64))
        classe_maj_inf = bincount_inf.argmax()
        classe_maj_sup = bincount_sup.argmax()        
    
    #compute overall metric value
    metric_data_inf = calculate_metric_3(data_inf, classe_maj_inf, idx_montant, idx_taux, idx_nbe)
    metric_data_sup = calculate_metric_3(data_sup, classe_maj_sup, idx_montant, idx_taux, idx_nbe)
    
    overall_metric= metric_data_inf + metric_data_sup
    
    return overall_metric

#Modified metric: Here the aim is to compute the total lost that we'll have
def modified_metric_3(data, idx_montant, idx_taux, idx_nbe):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = idx_montant #13
    index_taux = idx_taux #12
    index_nbe = idx_nbe #4
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        #else:
            #M += (data[line, index_taux]/100) * data[line,index_montant] 
    #print(P)
    return P

#Compute our overall modified metric
def overall_modified_metric_3(data_inf, data_sup, idx_montant, idx_taux, idx_nbe):
    #get number of datas
    data_all_lenght = len(data_inf) + len(data_sup)
    
    #compute overall metric value
    metric_data_inf = (len(data_inf) / data_all_lenght)*modified_metric_3(data_inf, idx_montant, idx_taux, idx_nbe)
    metric_data_sup = (len(data_sup) / data_all_lenght)*modified_metric_3(data_sup, idx_montant, idx_taux, idx_nbe)
    overall_metric= metric_data_inf + metric_data_sup
    #print("({}, {})".format(metric_data_inf, metric_data_sup))
    
    return overall_metric

#parralelized function
def computing_best_column_3(data, colum_index, value, idx_montant, idx_taux, idx_nbe):
    
    global best_split_column, best_split_value, overall_metric_value
    
    #print("current column: {}".format(colum_index))
    data_inf, data_sup = split_data_3(data, colum_index, value)
    current_overall = overall_modified_metric_3(data_inf, data_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
    if current_overall <= overall_metric_value:
        ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
        overall_metric_value = current_overall
        best_split_column = colum_index
        best_split_value = value
        #print("Done !! bests: ({}, {}, {})".format(best_split_column, best_split_value, overall_metric_value))

#determine best split attribute and value
def determine_best_split_3(data, potential_splits, idx_montant, idx_taux, idx_nbe):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    #global best_split_column, best_split_value, overall_metric_value
    
    #overall_metric_value = 300000000000000
    overall_metric_value = float('inf')
    #with Parallel(n_jobs=-1, backend="threading", require="sharedmem", verbose=5) as parallel:
      #  parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    best_split_columns = 0
    best_split_value = 0
    for colum_index in potential_splits:
        #print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            #print("current value: {}".format(value))
            data_inf, data_sup = split_data_3(data, colum_index, value)
            current_overall = overall_metric_3(data_inf, data_sup, idx_montant, idx_taux, idx_nbe)
            #print(current_overall)
            #check if lower
            if current_overall <= overall_metric_value:
                ####print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
                overall_metric_value = current_overall
                best_split_columns = colum_index
                best_split_value = value
    #print(best_split_value)
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    #print("Final Done !! bests: ({}, {}, {})".format(best_split_columns, best_split_value, overall_metric_value))
    return best_split_columns, best_split_value, overall_metric_value

#building decision Tree
def decision_tree_3(df, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter=0, min_samples=5, max_depth=5, metric="entropy"):
    
    if counter == 0:
        if isinstance(df, pd.DataFrame):
            data = df.values
        else: 
            data = df
        global COLUMNS_NAMES, FEATURE_TYPES, METRIC
        COLUMNS_NAMES = df.columns[:-1]
        FEATURE_TYPES = determine_feature_types(df)
        METRIC = metric
    else:
        data = df
        
    #base case
    potential_splits = get_splits(data, index_feature_not_selected, idx_montant)
    #print(potential_splits)
    if check_purity(data) or (len(data) < min_samples) or (counter == max_depth) or potential_splits == {}:
        classification = modified_classify_3(data, idx_montant, idx_taux, idx_nbe)
        return classification
    
    else:
        counter +=1
        #computations for right and left part 
        best_split_column, best_split_value, overall_metric_value = determine_best_split_3(data, potential_splits, idx_montant, idx_taux, idx_nbe)
        
        #We must change data_inf & data_sup order later
        data_inf, data_sup = split_data_3(data, best_split_column, best_split_value)
        
        # Creating subTree
        feature_type = FEATURE_TYPES[best_split_column]    
        if feature_type == "Continous":
            question = "{} <= {}".format(COLUMNS_NAMES[best_split_column], best_split_value)
        else:
            question = "{} == {}".format(COLUMNS_NAMES[best_split_column], best_split_value)
        #Adding labels
        labels = " " + str(len(data_inf) + len(data_sup)) + " " + str(overall_metric_value)
        question += labels
        sub_tree = {question: []}
        ###print(question)            
        #index_feature_not_selected.append(best_split_column)
        #left and right
        yes_answer = decision_tree_3(data_inf, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC)
        no_answer = decision_tree_3(data_sup, index_feature_not_selected, idx_montant, idx_taux, idx_nbe, counter, min_samples, max_depth, metric=METRIC)
        
        if yes_answer == no_answer:
            ###print(yes_answer)
            sub_tree = yes_answer
        else:
            #Append left and right part
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree
        

In [6]:


def test_presence(classe_name, x):
    val = False
    for i in classe_name:
        if i in x:
            val = True
    return val

def processing_feature_selection_AR(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    vector = dict()
    for i in df.columns.values:
        vector[i] = []
    del vector[classe]

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        vector[item].append(rules['confidence'][i])
    print(vector)
    print("\n")
    for key, value in vector.items():
        if vector[key] == []:
            vector[key] = 0.0
        else:
            vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector



def processing_feature_selection_AR_base(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    vector = dict()
    
    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if not item in vector:
            vector[item] = []
        vector[item].append(rules['confidence'][i])
    
    for key, value in vector.items():
        vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector



def processing_feature_selection_AR_merge_normal_inverse(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = df[classe].unique()
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules1 = rules
    rules2 = rules
    
    rules1['select'] = rules1['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules1 = rules1[ rules1['select'] == True]
    rules1['select'] = rules1['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules1 = rules1[ rules1['select'] == True]
    
    rules1['length_antecedents'] = rules1['antecedents'].apply(lambda x: len(x))
    rules1['length_consequents'] = rules1['consequents'].apply(lambda x: len(x))
    rules1 = rules1[ (rules1['length_antecedents'] == 1) & (rules1['length_consequents'] == 1)]
    
    
    rules2['select'] = rules2['antecedents'].apply(lambda x: test_presence(classe_name, x))
    rules2 = rules2[ rules2['select'] == True]
    rules2['select'] = rules2['consequents'].apply(lambda x: not test_presence(classe_name, x))
    rules2 = rules2[ rules2['select'] == True]
    
    rules2['length_antecedents'] = rules2['antecedents'].apply(lambda x: len(x))
    rules2['length_consequents'] = rules2['consequents'].apply(lambda x: len(x))
    rules2 = rules2[ (rules2['length_antecedents'] == 1) & (rules2['length_consequents'] == 1)]                   
    
    vector1 = dict()
    vector2 = dict()
    for i in df.columns.values:
        vector1[i] = []
        vector2[i] = []
    del vector1[classe]
    del vector2[classe]
    
    for i in rules1.index:
        item = list(rules1['antecedents'][i])[0]
        item = item.split(sep)[0]
        vector1[item].append(rules1['confidence'][i])
    
    for i in rules2.index:    
        item = list(rules2['consequents'][i])[0]
        item = item.split(sep)[0]
        vector2[item].append(rules2['confidence'][i])
        
        #print(item)
    for key, value in vector1.items():
        if vector1[key] == []:
            vector1[key] = 0.0
        else:
            vector1[key] = np.max(value, axis=0)
    
    for key, value in vector2.items():
        if vector2[key] == []:
            vector2[key] = 0.0
        else:
            vector2[key] = np.max(value, axis=0)

    for key, value in vector1.items():
        vector1[key] = vector1[key] + vector2[key]

            
    sorted_vector = sorted(vector1.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector


def processing_feature_selection_AR_inverse(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = df[classe].unique()
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    
    rules['select'] = rules['antecedents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['consequents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    vector = dict()
    for i in df.columns.values:
        vector[i] = []
    del vector[classe]

    for i in rules.index:
        item = list(rules['consequents'][i])[0]
        item = item.split(sep)[0]
        vector[item].append(rules['confidence'][i])
        #print(item)
    for key, value in vector.items():
        if vector[key] == []:
            vector[key] = 0.0
        else:
            vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector




def processing_feature_selection_AR_oriented_classe(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    #classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[val_class].unique())
    for i in range(len(unique_classe)):
        unique_classe[i] = str(unique_classe[i]) 
    
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(val_class + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    columns = list(set(df.columns)-set([val_class]))

    bloc = dict()
    for i in unique_classe:
        bloc[i] = dict()
    
    for i in unique_classe:
        for j in columns:
            bloc[i][j] = []
    #print(bloc)    
    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        classe = list(rules['consequents'][i])[0]
        classe = classe.split(sep)[1]
        bloc[classe][item].append(rules['confidence'][i])
    
    #print(rules)
    for classe in unique_classe:
        for key, value in bloc[classe].items():
            if bloc[classe][key] == []:
                bloc[classe][key] = 0.0
            else:
                bloc[classe][key] = np.max(value, axis=0)
    
    
    vector = dict()    
    for i in columns:
        vector[i] = 0.0
        
    for i in columns:
        ls = []
        for j in unique_classe:
            ls.append(bloc[j][i])
        #ls_norm = 1 / (1 + np.exp(-np.array(ls)))
        ls_norm = expit(np.array(ls))
        vector[i] = gini_ev(ls_norm)
        #vector[i] = vector[i] + bloc[j][i]
    
    #for i in columns:
    #    if vector[i] == 0.0:
    #        vector[i] = 1.0
        
    #sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    
    return sorted_vector
def gini_ev(ls):
    s = 0.0
    for i in range(len(ls)):
        s = s + ls[i]**2
    return 1-s

def processing_feature_selection_AR_Inverse_oriented_classe(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    #classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[val_class].unique())
    for i in range(len(unique_classe)):
        unique_classe[i] = str(unique_classe[i]) 
    
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(val_class + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    
    rules['select'] = rules['antecedents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['consequents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    columns = list(set(df.columns)-set([val_class]))

    bloc = dict()
    for i in unique_classe:
        bloc[i] = dict()

    for i in unique_classe:
        for j in columns:
            bloc[i][j] = []
        
    for i in rules.index:
        item = list(rules['consequents'][i])[0]
        item = item.split(sep)[0]
        classe = list(rules['antecedents'][i])[0]
        classe = classe.split(sep)[1]
        bloc[classe][item].append(rules['confidence'][i])
    
    print(bloc)
    print("\n")
    for classe in unique_classe:
        for key, value in bloc[classe].items():
            if bloc[classe][key] == []:
                bloc[classe][key] = 0.0
            else:
                bloc[classe][key] = np.max(value, axis=0)
    
    
    vector = dict()    
    for i in columns:
        vector[i] = 0.0
        
    for i in columns:
        ls = []
        for j in unique_classe:
            ls.append(bloc[j][i])
        #ls_norm = 1 / (1 + np.exp(-np.array(ls)))
        ls_norm = expit(np.array(ls))
        vector[i] = gini_ev(ls_norm)
        #vector[i] = vector[i] + bloc[j][i]
    
    #for i in columns:
    #    if vector[i] == 0.0:
    #        vector[i] = 1.0
        
    #sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    print(bloc)
    print("\n")
    return sorted_vector
            
def apply_loss_rules(rules, data, sep, name_montant, name_taux, name_duree, name_classe):
    rules['confidence_loss'] = rules['confidence']
    for i in rules.index:
        datas = data.copy()
        item = list(rules['antecedents'][i])[0]
        item_a = item.split(sep)[0]
        item_b = item.split(sep)[1]
        datas['select'] = datas[item_a].apply(lambda x: str(x) == item_b)
        datas = datas[datas['select'] == True]
        
        #total_loss = 0.0
        #for j in datas.index:
        #    if datas[name_classe][j] == 0:
                 #print("tchuente 11")
        #        total_loss += datas[name_montant][j]
        #    else:
                #print("tchuente 12")
        #        total_loss += datas[name_montant][j] * (datas[name_taux][j]/100) * datas[name_duree][j]/12
        
        item = list(rules['consequents'][i])[0]
        item_a = item.split(sep)[0]
        item_b = item.split(sep)[1]
        datas['select'] = datas[item_a].apply(lambda x: str(x) == item_b)
        datas = datas[datas['select'] == True]
        
        #print(rules)

        if item_b == '0':
            #print("tchuente 1")
            loss = datas[name_montant].sum()
        else:
            #print("tchuente 2")
            df_no_win = datas[[name_montant, name_taux, name_duree]]
            df_no_win[name_taux] /= 100
            df_no_win["NoWin"] = df_no_win[name_montant] * df_no_win[name_taux] * df_no_win[name_duree]/12
            loss = df_no_win.NoWin.sum()

        rules['confidence_loss'][i] = rules['confidence'][i]*loss
        #rules['confidence_loss'][i] = loss
        
    return rules
        
        
        
def processing_feature_selection_AR_Sensitive_Loss(df, dfv, sep, val_class, name_montant, name_taux, name_duree, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)
    
    
    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    vector = dict()
    columns = list(df.columns.values)
    for i in columns:
        if i == name_montant:
            continue
        vector[i] = []
        
    del vector[classe]
    rules = apply_loss_rules(rules, df.copy(deep=True), sep, name_montant, name_taux, name_duree, val_class)

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if item == name_montant:
            continue
        vector[item].append(rules['confidence_loss'][i])
    
    print(vector)
    print("\n")
    for key, value in vector.items():
        if vector[key] == []:
            vector[key] = 0.0
        else:
            vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector




def processing_feature_selection_AR_Sensitive_Loss_(df, dfv, sep, val_class, name_montant, name_taux, name_duree, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)
    
    
    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    vector = dict()
    columns = list(df.columns.values)
    for i in columns:
        if i == name_montant:
            continue
        vector[i] = []
        
    del vector[classe]
    rules = apply_loss_rules(rules, df.copy(deep=True), sep, name_montant, name_taux, name_duree, val_class)

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if item == name_montant:
            continue
        vector[item].append(rules['confidence_loss'][i])
    
    print(vector)
    print("\n")
    for key, value in vector.items():
        if vector[key] == []:
            vector[key] = 0.0
        else:
            vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector


def processing_feature_selection_AR_base_Sensitive_Loss_(df, dfv, sep, val_class, name_montant, name_taux, name_duree, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)
    
    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    vector = dict()
    rules = apply_loss_rules(rules, df.copy(deep=True), sep, name_montant, name_taux, name_duree, val_class)

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if item == name_montant:
            continue
        if not item in vector:
            vector[item] = []
        vector[item].append(rules['confidence_loss'][i])
    
    for key, value in vector.items():
        vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=False)

    return sorted_vector




def processing_feature_selection_AR_base(df, dfv, sep, val_class, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)

    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    vector = dict()
    
    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if not item in vector:
            vector[item] = []
        vector[item].append(rules['confidence'][i])
    
    for key, value in vector.items():
        vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=True)

    return sorted_vector



def processing_feature_selection_AR_base_Loss_(df, dfv, sep, val_class, name_montant, name_taux, name_duree, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min)
    
    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)]                   
    
    vector = dict()
    rules = apply_loss_rules(rules, df.copy(deep=True), sep, name_montant, name_taux, name_duree, val_class)

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if item == name_montant:
            continue
        if not item in vector:
            vector[item] = []
        vector[item].append(rules['confidence_loss'][i])
    
    #for key, value in vector.items():
    #    vector[key] = np.max(value, axis=0)
    #sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=False)
    
    train, test = train_test_split_(df, 0.3)
    index_feature_not_selected = []
    idx_montant = df.columns.get_loc(name_montant)
    idx_taux = df.columns.get_loc(name_taux)
    idx_duree = df.columns.get_loc(name_duree)
    
    #random = 10
    cricterior = 'cs'
    set_feature = dict()
    columns = df.columns.tolist()
    len_columns = len(columns)
    columns = list(set(columns) - set([val_class]))
    #X_train, X_test, y_train, y_test, columns_extract_all = preprocessing_dataset_(df, [val_class], sep, val_class, random)
    for col in columns:
        if not col in vector:       
            #X_train_selected, X_test_selected, columns_extract = selected_dataset_(columns_extract_all, [col], sep, X_train, X_test)
            #means_scores, _, _, _ = model_evaluation_SFS_(X_train_selected, X_test_selected, y_train, y_test, columns_extract, columns, sep, cricterior)
            #set_feature[col] = means_scores
            
            index_feature_not_selected = list(set(range(len_columns)) - set([df.columns.get_loc(col)]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([df.columns.get_loc(val_class)]))
        
            my_tree = decision_tree(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy, df_p = my_accuracy(test, my_tree, val_class)
            _, means_scores = compute_metrics(df_p, val_class, name_montant, name_taux, name_duree)
            set_feature[col] = means_scores
            
    for key, value in vector.items():
        vector[key] = np.max(value, axis=0)
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=False)
    set_feature = sorted(set_feature.items(), key=lambda x:x[1], reverse=False)
    print("sorted_vector : ", sorted_vector)
    print("set_feature : ", set_feature)
    for ele in set_feature:
        sorted_vector.append(ele)

    return sorted_vector







def processing_feature_selection_AR_base_Multi_Loss(df, dfv, cricterior, alpha, sep, val_class, name_montant, name_taux, name_duree, support_min_loss=0.07, metric_v="confidence", threshold_min_loss=0.4, random=10):
    #dict_loss = give_dict_loss(df, name_montant)
    #print("dict_loss : ", dict_loss)
    #frequent_items_loss = apriori_loss_sensitive(dfv.astype('bool'), dict_loss, min_support_loss=support_min_loss)
    #print("frequent_items_loss : ", frequent_items_loss)
    #rules = association_rules_(frequent_items_loss, metric_v, threshold_min_loss)
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min_loss, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min_loss)
    
    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)] 
    #print(rules)
    
    vector = dict()
    rules = apply_loss_rules(rules, df.copy(deep=True), sep, name_montant, name_taux, name_duree, val_class)

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if item == name_montant:
            continue
        if not item in vector:
            vector[item] = []
        #vector[item].append(rules['confidence_loss'][i])
        vector[item].append(rules['confidence_loss'][i])
    
    dataset_y = df[val_class]
    dataset_x = df.drop([val_class], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random)
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    index_feature_not_selected = []
    idx_montant = df.columns.get_loc(name_montant)
    idx_taux = df.columns.get_loc(name_taux)
    idx_duree = df.columns.get_loc(name_duree)
    
    #random = 10
    cricterior = cricterior
    set_feature = dict()
    columns = df.columns.tolist()
    len_columns = len(columns)
    columns = list(set(columns) - set([val_class]))
    #X_train, X_test, y_train, y_test, columns_extract_all = preprocessing_dataset_(df, [val_class], sep, val_class, random)
    for col in columns:
        if not col in vector:       
            #X_train_selected, X_test_selected, columns_extract = selected_dataset_(columns_extract_all, [col], sep, X_train, X_test)
            #means_scores, _, _, _ = model_evaluation_SFS_(X_train_selected, X_test_selected, y_train, y_test, columns_extract, columns, sep, cricterior)
            #set_feature[col] = means_scores
            
            index_feature_not_selected = list(set(range(len_columns)) - set([df.columns.get_loc(col)]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([df.columns.get_loc(val_class)]))
        
            my_tree = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy, df_p = my_accuracy(test, my_tree, val_class)
            _, means_scores = compute_metrics(df_p, val_class, name_montant, name_taux, name_duree)
            set_feature[col] = means_scores
            
    for key, value in vector.items():
        vector[key] = np.max(value, axis=0)
    
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=False)
    set_feature = sorted(set_feature.items(), key=lambda x:x[1], reverse=False)
    print("sorted_vector : ", sorted_vector)
    print("set_feature : ", set_feature)
    for ele in set_feature:
        sorted_vector.append(ele)

    return sorted_vector





def processing_feature_selection_AR_base_Multi_Loss_(df, dfv, cricterior, alpha, sep, val_class, name_montant, name_taux, name_duree, support_min_loss=0.07, metric_v="confidence", threshold_min_loss=0.4, random=10, decision_tree_cs="decision_tree"):
    #dict_loss = give_dict_loss(df, name_montant)
    #print("dict_loss : ", dict_loss)
    #frequent_items_loss = apriori_loss_sensitive(dfv.astype('bool'), dict_loss, min_support_loss=support_min_loss)
    #print("frequent_items_loss : ", frequent_items_loss)
    #rules = association_rules_(frequent_items_loss, metric_v, threshold_min_loss)
    frequent_itemsets = apriori(dfv.astype('bool'), min_support=support_min_loss, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric=metric_v, min_threshold=threshold_min_loss)
    
    classe = val_class
    #classe = df.columns.values[0]
    unique_classe = list(df[classe].unique())
    classe_name = []
    for i in range(len(unique_classe)):
        classe_name.append(classe + sep + str(unique_classe[i]))
    
    #rules['select'] = rules['consequents'].apply(lambda x: 'X1_0' in x or 'X1_1' in x)
    rules['select'] = rules['consequents'].apply(lambda x: test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    rules['select'] = rules['antecedents'].apply(lambda x: not test_presence(classe_name, x))
    rules = rules[ rules['select'] == True]
    
    rules['length_antecedents'] = rules['antecedents'].apply(lambda x: len(x))
    rules['length_consequents'] = rules['consequents'].apply(lambda x: len(x))
    rules = rules[ (rules['length_antecedents'] == 1) & (rules['length_consequents'] == 1)] 
    #print(rules)
    
    vector = dict()
    rules = apply_loss_rules(rules, df.copy(deep=True), sep, name_montant, name_taux, name_duree, val_class)

    for i in rules.index:
        item = list(rules['antecedents'][i])[0]
        item = item.split(sep)[0]
        if item == name_montant:
            continue
        if not item in vector:
            vector[item] = []
        #vector[item].append(rules['confidence_loss'][i])
        vector[item].append(rules['confidence_loss'][i])
    
    dataset_y = df[val_class]
    dataset_x = df.drop([val_class], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random)
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    index_feature_not_selected = []
    idx_montant = df.columns.get_loc(name_montant)
    idx_taux = df.columns.get_loc(name_taux)
    idx_duree = df.columns.get_loc(name_duree)
    
    #random = 10
    cricterior = cricterior
    set_feature = dict()
    columns = df.columns.tolist()
    len_columns = len(columns)
    columns = list(set(columns) - set([val_class]))
    #X_train, X_test, y_train, y_test, columns_extract_all = preprocessing_dataset_(df, [val_class], sep, val_class, random)
    for col in columns:
        if not col in vector:       
            #X_train_selected, X_test_selected, columns_extract = selected_dataset_(columns_extract_all, [col], sep, X_train, X_test)
            #means_scores, _, _, _ = model_evaluation_SFS_(X_train_selected, X_test_selected, y_train, y_test, columns_extract, columns, sep, cricterior)
            #set_feature[col] = means_scores
            
            index_feature_not_selected = list(set(range(len_columns)) - set([df.columns.get_loc(col)]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([df.columns.get_loc(val_class)]))
        
            my_tree = decision_tree_cs(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy, df_p = my_accuracy(test, my_tree, val_class)
            _, means_scores = compute_metrics(df_p, val_class, name_montant, name_taux, name_duree)
            set_feature[col] = means_scores
            
    for key, value in vector.items():
        vector[key] = np.max(value, axis=0)
    
    sorted_vector = sorted(vector.items(), key=lambda x:x[1], reverse=False)
    set_feature = sorted(set_feature.items(), key=lambda x:x[1], reverse=False)
    print("sorted_vector : ", sorted_vector)
    print("set_feature : ", set_feature)
    for ele in set_feature:
        sorted_vector.append(ele)

    return sorted_vector




In [7]:
def model_evaluation_SFS(X_train, X_test, y_train, y_test, criterior='entropy'):
    model = DecisionTreeClassifier(criterion=criterior, random_state=0)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    X_data = np.concatenate((X_train, X_test), axis=0)
    y_data = np.concatenate((y_train, y_test), axis=0)
    scores = cross_val_score(model, X=X_data, y=y_data, cv=10, n_jobs=1)
    
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
    #print("Confusion Matrix : ", conf_matrix)
    prec = precision_score(y_test, y_pred)
    
    means_scores = np.mean(scores)
    
    return means_scores, conf_matrix, prec


def feature_selection_SFS_ARFS_Sensitive_Loss(dataset, dfv, sep, name_columns_drop, name_classe, name_montant, name_taux, name_duree, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    #dataset, y_train = get_dataset_classe()
    
    divide = True
    i = 0
    j = 0
    max_scores = float('inf')
    sorted_vector = processing_feature_selection_AR_Sensitive_Loss(dataset, dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    
    beta = len(sorted_vector)
    opti_feature = dict()
    suite_index = dict()
    k = 0
    cricterior = 'cs'
    feature_vector = dict()
    itera = 1
    feature_vector[j] = []
    train, test = train_test_split_(dataset, 0.3)
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    while divide == True:
        feature_vector[j].append(sorted_vector[i][0])
        #print(feature_vector[j])
        ###data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]
        
        #X_train, X_test, y_train, y_test = preprocessing_dataset(data, name_columns_drop, name_classe)
        #means_scores, conf_matrix, prec = model_evaluation_SFS(X_train, X_test, y_train, y_test, cricterior)
        
        
        index_feature_not_selected = list(set(range(len(dataset.columns.values.tolist()))) - set([dataset.columns.get_loc(col) for col in feature_vector[j]])) 
        index_feature_not_selected = list(set(index_feature_not_selected) - set([name_classe])) 
        
        my_tree = decision_tree(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
        accuracy, df = my_accuracy(test, my_tree, name_classe)
        _, means_scores = compute_metrics(df, name_classe, name_montant, name_taux, name_duree)
        #Use prepocessing data in X_train to transform the data before the training at the model 
        #model.fit(X_train, y_train)
        #scores = cross_val_score(model, X=X_train, y=y_train, cv=10, n_jobs=1)
        #means_scores = np.mean(scores)
        
        if means_scores < max_scores:
            max_scores = means_scores
            opt_tree = my_tree
            opti_feature[i] = [deepcopy(feature_vector[j]), means_scores, len(feature_vector[j]), accuracy] 
        else:
            if means_scores == max_scores:
                feature_vector[itera] = deepcopy(feature_vector[j])
                suite_index[itera] = i
                itera = itera + 1
            else:
                feature_vector[j].pop(len(feature_vector[j])-1)
        #print(opti_feature)
        if i >= beta-1:
            if j < len(feature_vector)-1:
                j = j + 1
                i = suite_index[j]                
                if i >= beta-1:
                    divide = False
            else:
                divide = False
        i = i + 1
    return opti_feature, max_scores, sorted_vector, opt_tree





def feature_selection_SFS_ARFS_Sensitive_Loss_(dataset, dfv, sep, name_columns_drop, name_classe, name_montant, name_taux, name_duree, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    divide = True
    i = 0
    j = 0
    train, test = train_test_split_(dataset, 0.3)
    #sorted_vector = processing_feature_selection_AR_base_Sensitive_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    sorted_vector = processing_feature_selection_AR_base_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    
    beta = len(sorted_vector)
    opti_feature = dict()
    suite_index = dict()
    k = 0
    opti_accuracy = 0.0
    cricterior = 'cs'
    feature_vector = dict()
    itera = 1
    feature_vector[j] = []
    
    #print(np.unique(train[:,7], return_counts = True))
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    max_scores = float('inf')
    columns = dataset.columns.tolist()
    len_columns = len(columns)
    while divide == True:
        feature_vector[j].append(sorted_vector[i][0])
        ####print("feature_vector[j] : ", feature_vector[j])
        #print(feature_vector[j])
        ###data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]
        
        #X_train, X_test, y_train, y_test = preprocessing_dataset(data, name_columns_drop, name_classe)
        #means_scores, conf_matrix, prec = model_evaluation_SFS(X_train, X_test, y_train, y_test, cricterior)
        
        index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j]]))
        index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)])) 
        
        my_tree = decision_tree(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
        accuracy, df = my_accuracy(test, my_tree, name_classe)
        _, means_scores = compute_metrics(df, name_classe, name_montant, name_taux, name_duree)
        
        ####print("means_scores : ", means_scores)
        ####print("my_tree : ", my_tree)
        #Use prepocessing data in X_train to transform the data before the training at the model 
        #model.fit(X_train, y_train)
        #scores = cross_val_score(model, X=X_train, y=y_train, cv=10, n_jobs=1)
        #means_scores = np.mean(scores)
        #print("my_tree :", my_tree)
        #print("max_scores : ", max_scores, "; Accuracy : ", accuracy, " ; means_scores : ", means_scores, "; feature_vector[j] : ", feature_vector[j], "; index_feature_not_selected: ", index_feature_not_selected)
        if means_scores < max_scores:
            #print("tchuente")
            max_scores = means_scores
            opt_tree = my_tree
            #opti_accuracy = accuracy
            feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
            ####print("feature_vector[j]& : ", feature_vector[j])
            #print("feature_vector[j]-optimal: ", feature_vector[j])
            opti_feature[i] = [deepcopy(feature_vector[j]), means_scores, len(feature_vector[j]), accuracy] 
        else:
            if means_scores == max_scores:
                feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                feature_vector[itera] = deepcopy(feature_vector[j])
                suite_index[itera] = i
                itera = itera + 1
            else:
                feature_vector[j].pop(len(feature_vector[j])-1)
        #print(opti_feature)
        if i >= beta-1:
            if j < len(feature_vector)-1:
                j = j + 1
                i = suite_index[j]                
                if i >= beta-1:
                    divide = False
            else:
                divide = False
        i = i + 1
    index_feature_not_selected = []
    my_tree = decision_tree(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
    accuracy, df = my_accuracy(test, my_tree, name_classe)
    results_all_data = compute_metrics(df, name_classe, name_montant, name_taux, name_duree), extract_feature_select_tree(my_tree)    
    
    return opti_feature, max_scores, sorted_vector, opt_tree, results_all_data









def feature_selection_SFS_ARFS_Sensitive_Multi_Loss(dataset, dfv, sep, type_cout, alpha, name_columns_drop, name_classe, name_montant,
                                                    name_taux, name_duree, cricterior, support_min=0.07, metric_v="confidence", threshold_min=0.4):
    divide = True
    i = 0
    j = 0
    train, test = train_test_split_(dataset, 0.3)
    #sorted_vector = processing_feature_selection_AR_base_Sensitive_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_base_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    sorted_vector = processing_feature_selection_AR_base_Multi_Loss(dataset.copy(), dfv, cricterior, alpha, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    
    beta = len(sorted_vector)
    opti_feature = dict()
    suite_index = dict()
    k = 0
    opti_accuracy = 0.0
    cricterior = cricterior
    feature_vector = dict()
    itera = 1
    feature_vector[j] = []
    
    #print(np.unique(train[:,7], return_counts = True))
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    optimisation_minimisation = ["pessimiste"]
    optimisation_maximisation = ["total", "optimiste", "reel"]
    
    if type_cout in optimisation_minimisation:
        max_scores = float('inf')
    elif type_cout in optimisation_maximisation:
        max_scores = -float('inf')
        
    columns = dataset.columns.tolist()
    len_columns = len(columns)
    opt_tree = []
    
    while divide == True:
        feature_vector[j].append(sorted_vector[i][0])
        #print("feature_vector[j] : ", feature_vector[j])
        #print(feature_vector[j])
        ###data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]
        
        #X_train, X_test, y_train, y_test = preprocessing_dataset(data, name_columns_drop, name_classe)
        #means_scores, conf_matrix, prec = model_evaluation_SFS(X_train, X_test, y_train, y_test, cricterior)
        
        index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j]]))
        index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)])) 
        
        my_tree = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
        accuracy, df = my_accuracy(test, my_tree, name_classe)
        #_, means_scores = compute_metrics(df, name_classe, name_montant, name_taux, name_duree)
        _, means_scores = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
        
        #print("means_scores : ", means_scores)
        #print("my_tree : ", my_tree)
        #Use prepocessing data in X_train to transform the data before the training at the model 
        #model.fit(X_train, y_train)
        #scores = cross_val_score(model, X=X_train, y=y_train, cv=10, n_jobs=1)
        #means_scores = np.mean(scores)
        #print("my_tree :", my_tree)
        #print("max_scores : ", max_scores, "; Accuracy : ", accuracy, " ; means_scores : ", means_scores, "; feature_vector[j] : ", feature_vector[j], "; index_feature_not_selected: ", index_feature_not_selected)
        if type_cout in optimisation_minimisation:
            if means_scores < max_scores:
                #print("tchuente")
                max_scores = means_scores
                opt_tree = my_tree
                #opti_accuracy = accuracy
                if type(my_tree) is dict:    
                    feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                #print("feature_vector[j]& : ", feature_vector[j])
                #print("feature_vector[j]-optimal: ", feature_vector[j])
                opti_feature[i] = [deepcopy(feature_vector[j]), means_scores, len(feature_vector[j]), accuracy] 
            else:
                if means_scores == max_scores:
                    if type(my_tree) is dict:    
                        feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    feature_vector[itera] = deepcopy(feature_vector[j])
                    suite_index[itera] = i
                    itera = itera + 1
                else:
                    feature_vector[j].pop(len(feature_vector[j])-1)
        elif type_cout in optimisation_maximisation:
            if means_scores > max_scores:
                #print("tchuente")
                max_scores = means_scores
                opt_tree = my_tree
                #opti_accuracy = accuracy
                if type(my_tree) is dict:    
                    feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                #####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                ####print("feature_vector[j]& : ", feature_vector[j])
                #print("feature_vector[j]-optimal: ", feature_vector[j])
                opti_feature[i] = [deepcopy(feature_vector[j]), means_scores, len(feature_vector[j]), accuracy] 
            else:
                if means_scores == max_scores:
                    ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    if type(my_tree) is dict:    
                        feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    feature_vector[itera] = deepcopy(feature_vector[j])
                    suite_index[itera] = i
                    itera = itera + 1
                else:
                    feature_vector[j].pop(len(feature_vector[j])-1)
            
        #print(opti_feature)
        if i >= beta-1:
            if j < len(feature_vector)-1:
                j = j + 1
                i = suite_index[j]                
                if i >= beta-1:
                    divide = False
            else:
                divide = False
        i = i + 1
    index_feature_not_selected = []
    my_tree = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
    accuracy, df = my_accuracy(test, my_tree, name_classe)
    results_all_data = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree), extract_feature_select_tree(my_tree)    
    return opti_feature, max_scores, sorted_vector, opt_tree, results_all_data











def feature_selection_SFS_ARFS_Sensitive_Multi_Loss_(dataset, dfv, sep, type_cout, alpha, name_columns_drop, name_classe, name_montant,
                                                    name_taux, name_duree, cricterior, support_min=0.07, metric_v="confidence", threshold_min=0.4, random=10):
    divide = True
    i = 0
    j = 0
    
    dataset_y = dataset[name_classe]
    dataset_x = dataset.drop(name_columns_drop, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random)
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    
    #sorted_vector = processing_feature_selection_AR_base_Sensitive_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_base_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    sorted_vector = processing_feature_selection_AR_base_Multi_Loss(dataset.copy(), dfv, cricterior, alpha, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min, random)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    
    beta = len(sorted_vector)
    opti_feature = dict()
    suite_index = dict()
    k = 0
    opti_accuracy = 0.0
    cricterior = cricterior
    feature_vector = dict()
    itera = 1
    feature_vector[j] = deepcopy([deepcopy([]), deepcopy([])])    
    #print(np.unique(train[:,7], return_counts = True))
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    optimisation_minimisation = ["pessimiste"]
    optimisation_maximisation = ["total", "optimiste", "reel"]
    
    if type_cout in optimisation_minimisation:
        max_scores = float('inf')
    elif type_cout in optimisation_maximisation:
        max_scores = -float('inf')
    max_scores_acc = -float('inf')    
    columns = dataset.columns.tolist()
    len_columns = len(columns)
    opt_tree = []
    
    while divide == True:
        feature_vector[j][0].append(sorted_vector[i][0])
        feature_vector[j][1].append(sorted_vector[i][0])
        #print("feature_vector[j] : ", feature_vector[j])
        #print(feature_vector[j])
        ###data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]
        
        #X_train, X_test, y_train, y_test = preprocessing_dataset(data, name_columns_drop, name_classe)
        #means_scores, conf_matrix, prec = model_evaluation_SFS(X_train, X_test, y_train, y_test, cricterior)
        
        #index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j]]))
        #index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)])) 

        if feature_vector[j][0] != []:
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][0]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            my_tree_gen = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy_gen, df_gen = my_accuracy(test, my_tree_gen, name_classe)
            result_gen, means_scores_gen = compute_metrics_derive(df_gen, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            #print("index_feature_not_selected : ", index_feature_not_selected)
            #print("feature_vector[j][0] : ", feature_vector[j][0])
            #print("accuracy_sel : ", means_scores_gen)

        
        if feature_vector[j][1] != []:
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][1]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            my_tree_sel = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy_sel, df_sel = my_accuracy(test, my_tree_sel, name_classe)
            result_sel, means_scores_sel = compute_metrics_derive(df_sel, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            
        #print("means_scores : ", means_scores)
        #print("my_tree : ", my_tree)
        #Use prepocessing data in X_train to transform the data before the training at the model 
        #model.fit(X_train, y_train)
        #scores = cross_val_score(model, X=X_train, y=y_train, cv=10, n_jobs=1)
        #means_scores = np.mean(scores)
        #print("my_tree :", my_tree)
        #print("max_scores : ", max_scores, "; Accuracy : ", accuracy, " ; means_scores : ", means_scores, "; feature_vector[j] : ", feature_vector[j], "; index_feature_not_selected: ", index_feature_not_selected)
                
        
        if type_cout in optimisation_minimisation:
            if means_scores_sel <= means_scores_gen:
                means_scores = means_scores_sel
                accuracy = accuracy_sel
                my_tree = my_tree_sel
                result = result_sel
            else:
                means_scores = means_scores_gen
                accuracy = accuracy_gen
                my_tree = my_tree_gen
                result = result_gen

            if means_scores < max_scores:#if means_scores < max_scores and accuracy > max_scores_acc:
                #print("tchuente")
                max_scores = means_scores
                opt_tree = my_tree
                max_scores_acc = accuracy
                #opti_accuracy = accuracy
                if type(my_tree) is dict:    
                    feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                #print("feature_vector[j]& : ", feature_vector[j])
                #print("feature_vector[j]-optimal: ", feature_vector[j])
                opti_feature[i] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result] 
            else:
                if means_scores == max_scores:#if means_scores == max_scores:
                    if type(my_tree) is dict:    
                        feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                    ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    feature_vector[itera] = deepcopy([deepcopy(feature_vector[j][0]), deepcopy(feature_vector[j][1])])
                    suite_index[itera] = i
                    itera = itera + 1
                else:
                    #feature_vector[j][0].pop(len(feature_vector[j][0])-1)
                    feature_vector[j][1].pop(len(feature_vector[j][1])-1)
        elif type_cout in optimisation_maximisation:
            if means_scores_sel >= means_scores_gen:
                means_scores = means_scores_sel
                accuracy = accuracy_sel
                my_tree = my_tree_sel
                result = result_sel
            else:
                means_scores = means_scores_gen
                accuracy = accuracy_gen
                my_tree = my_tree_gen
                result = result_gen

            if means_scores > max_scores:#if means_scores > max_scores and accuracy > max_scores_acc:
                #print("tchuente")
                max_scores = means_scores
                max_scores_acc = accuracy
                opt_tree = my_tree
                #opti_accuracy = accuracy
                if type(my_tree) is dict:    
                    feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                #####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                ####print("feature_vector[j]& : ", feature_vector[j])
                #print("feature_vector[j]-optimal: ", feature_vector[j])
                opti_feature[i] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result] 
            else:
                if means_scores == max_scores:
                    ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    if type(my_tree) is dict:    
                        feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                    feature_vector[itera] = deepcopy([deepcopy(feature_vector[j][0]), deepcopy(feature_vector[j][1])])
                    suite_index[itera] = i
                    itera = itera + 1
                else:
                    #feature_vector[j][0].pop(len(feature_vector[j][0])-1)
                    feature_vector[j][1].pop(len(feature_vector[j][1])-1)
            
        #print(opti_feature)
        if i >= beta-1:
            if j < len(feature_vector)-1:
                j = j + 1
                i = suite_index[j]                
                if i >= beta-1:
                    divide = False
            else:
                divide = False
        i = i + 1
    index_feature_not_selected = []
    my_tree = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
    accuracy, df = my_accuracy(test, my_tree, name_classe)
    results_all_data = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree), extract_feature_select_tree(my_tree)    
    return opti_feature, max_scores, sorted_vector, opt_tree, results_all_data



In [8]:
def feature_selection_SFS_ARFS_Sensitive_Multi_Loss_(dataset, dfv, sep, type_cout, alpha, name_columns_drop, name_classe, name_montant,
                                                    name_taux, name_duree, cricterior, support_min=0.07, metric_v="confidence", threshold_min=0.4, random=10):
    divide = True
    i = 0
    j = 0
    
    dataset_y = dataset[name_classe]
    dataset_x = dataset.drop(name_columns_drop, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random)
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    
    #sorted_vector = processing_feature_selection_AR_base_Sensitive_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_base_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    sorted_vector = processing_feature_selection_AR_base_Multi_Loss(dataset.copy(), dfv, cricterior, alpha, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min, random)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    
    beta = len(sorted_vector)
    opti_feature = dict()
    suite_index = dict()
    k = 0
    opti_accuracy = 0.0
    cricterior = cricterior
    feature_vector = dict()
    itera = 1
    feature_vector[j] = deepcopy([deepcopy([]), deepcopy([])])    
    #print(np.unique(train[:,7], return_counts = True))
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    optimisation_minimisation = ["pessimiste"]
    optimisation_maximisation = ["total", "optimiste", "reel"]
    
    if type_cout in optimisation_minimisation:
        max_scores = float('inf')
    elif type_cout in optimisation_maximisation:
        max_scores = -float('inf')
    max_scores_acc = -float('inf')    
    columns = dataset.columns.tolist()
    len_columns = len(columns)
    opt_tree = []
    
    while divide == True:
        feature_vector[j][0].append(sorted_vector[i][0])
        feature_vector[j][1].append(sorted_vector[i][0])
        #print("feature_vector[j] : ", feature_vector[j])
        #print(feature_vector[j])
        ###data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]
        
        #X_train, X_test, y_train, y_test = preprocessing_dataset(data, name_columns_drop, name_classe)
        #means_scores, conf_matrix, prec = model_evaluation_SFS(X_train, X_test, y_train, y_test, cricterior)
        
        #index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j]]))
        #index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)])) 

        if feature_vector[j][0] != []:
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][0]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            my_tree_gen = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy_gen, df_gen = my_accuracy(test, my_tree_gen, name_classe)
            result_gen, means_scores_gen = compute_metrics_derive(df_gen, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            #print("index_feature_not_selected : ", index_feature_not_selected)
            #print("feature_vector[j][0] : ", feature_vector[j][0])
            #print("accuracy_sel : ", means_scores_gen)

        
        if feature_vector[j][1] != []:
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][1]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            my_tree_sel = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy_sel, df_sel = my_accuracy(test, my_tree_sel, name_classe)
            result_sel, means_scores_sel = compute_metrics_derive(df_sel, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            
        #print("means_scores : ", means_scores)
        #print("my_tree : ", my_tree)
        #Use prepocessing data in X_train to transform the data before the training at the model 
        #model.fit(X_train, y_train)
        #scores = cross_val_score(model, X=X_train, y=y_train, cv=10, n_jobs=1)
        #means_scores = np.mean(scores)
        #print("my_tree :", my_tree)
        #print("max_scores : ", max_scores, "; Accuracy : ", accuracy, " ; means_scores : ", means_scores, "; feature_vector[j] : ", feature_vector[j], "; index_feature_not_selected: ", index_feature_not_selected)
                
        
        if type_cout in optimisation_minimisation:
            if means_scores_sel <= means_scores_gen:
                means_scores = means_scores_sel
                accuracy = accuracy_sel
                my_tree = my_tree_sel
                result = result_sel
            else:
                means_scores = means_scores_gen
                accuracy = accuracy_gen
                my_tree = my_tree_gen
                result = result_gen

            if means_scores < max_scores:#if means_scores < max_scores and accuracy > max_scores_acc:
                #print("tchuente")
                max_scores = means_scores
                opt_tree = my_tree
                max_scores_acc = accuracy
                #opti_accuracy = accuracy
                if type(my_tree) is dict:    
                    feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                #print("feature_vector[j]& : ", feature_vector[j])
                #print("feature_vector[j]-optimal: ", feature_vector[j])
                opti_feature[i] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result] 
            else:
                if means_scores == max_scores:#if means_scores == max_scores:
                    if type(my_tree) is dict:    
                        feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                    ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    feature_vector[itera] = deepcopy([deepcopy(feature_vector[j][0]), deepcopy(feature_vector[j][1])])
                    suite_index[itera] = i
                    itera = itera + 1
                else:
                    #feature_vector[j][0].pop(len(feature_vector[j][0])-1)
                    feature_vector[j][1].pop(len(feature_vector[j][1])-1)
        elif type_cout in optimisation_maximisation:
            if means_scores_sel >= means_scores_gen:
                means_scores = means_scores_sel
                accuracy = accuracy_sel
                my_tree = my_tree_sel
                result = result_sel
            else:
                means_scores = means_scores_gen
                accuracy = accuracy_gen
                my_tree = my_tree_gen
                result = result_gen

            if means_scores > max_scores:#if means_scores > max_scores and accuracy > max_scores_acc:
                #print("tchuente")
                max_scores = means_scores
                max_scores_acc = accuracy
                opt_tree = my_tree
                #opti_accuracy = accuracy
                if type(my_tree) is dict:    
                    feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                #####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                ####print("feature_vector[j]& : ", feature_vector[j])
                #print("feature_vector[j]-optimal: ", feature_vector[j])
                opti_feature[i] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result] 
            else:
                if means_scores == max_scores:
                    ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    if type(my_tree) is dict:    
                        feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                    feature_vector[itera] = deepcopy([deepcopy(feature_vector[j][0]), deepcopy(feature_vector[j][1])])
                    suite_index[itera] = i
                    itera = itera + 1
                else:
                    #feature_vector[j][0].pop(len(feature_vector[j][0])-1)
                    feature_vector[j][1].pop(len(feature_vector[j][1])-1)
            
        #print(opti_feature)
        if i >= beta-1:
            if j < len(feature_vector)-1:
                j = j + 1
                i = suite_index[j]                
                if i >= beta-1:
                    divide = False
            else:
                divide = False
        i = i + 1
    index_feature_not_selected = []
    my_tree = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
    accuracy, df = my_accuracy(test, my_tree, name_classe)
    results_all_data = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree), extract_feature_select_tree(my_tree)    
    return opti_feature, max_scores, sorted_vector, opt_tree, results_all_data



In [9]:
def delete_element_list(list_element_at_remove, set_element):
    print("list_element_at_remove : ", list_element_at_remove)
    print("set_element : ", set_element)
    for i in range(len(list_element_at_remove)):
        delete = False
        j = 0
        while delete == False and j < len(set_element):
            if list_element_at_remove[i] == set_element[j][0]:
                delete = True
                set_element.remove(set_element[j])
            else:
                j = j + 1
    print("set_element results : ", set_element)
    return set_element


def feature_selection_SFS_ARFS_Sensitive_Multi_Loss_Boosting(dataset, dfv, sep, type_cout, alpha, name_columns_drop, name_classe, name_montant,
                                                    name_taux, name_duree, cricterior, nb_estimator = 5, support_min=0.07, metric_v="confidence", threshold_min=0.4, random=10):
    divide = True
    i = 0
    j = 0
    
    dataset_y = dataset[name_classe]
    dataset_x = dataset.drop(name_columns_drop, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random)
    train = pd.concat([X_train, y_train], axis=1)
    test = pd.concat([X_test, y_test], axis=1)
    
    
    #sorted_vector = processing_feature_selection_AR_base_Sensitive_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    #sorted_vector = processing_feature_selection_AR_base_Loss_(dataset.copy(), dfv, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min)
    sorted_vector = processing_feature_selection_AR_base_Multi_Loss(dataset.copy(), dfv, cricterior, alpha, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min, random)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    
    beta = len(sorted_vector)
    opti_feature = dict()
    opti_feature_boost = dict()
    list_of_opti_feature = deepcopy([])
    suite_index = dict()
    k = 0
    opti_accuracy = 0.0
    cricterior = cricterior
    feature_vector = dict()
    itera = 1
    feature_vector[j] = deepcopy([deepcopy([]), deepcopy([])])
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    optimisation_minimisation = ["pessimiste"]
    optimisation_maximisation = ["total", "optimiste", "reel"]
    
    if type_cout in optimisation_minimisation:
        max_scores = float('inf')
    elif type_cout in optimisation_maximisation:
        max_scores = -float('inf')
    max_scores_acc = -float('inf')    
    columns = dataset.columns.tolist()
    len_columns = len(columns)
    opt_tree = deepcopy([])
    index_opti_feature = None
    for estimator_i in range(nb_estimator):
        while divide == True:
            feature_vector[j][0].append(sorted_vector[i][0])
            feature_vector[j][1].append(sorted_vector[i][0])
            #print("feature_vector[j] : ", feature_vector[j])
            #print(feature_vector[j])
            ###data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]

            #X_train, X_test, y_train, y_test = preprocessing_dataset(data, name_columns_drop, name_classe)
            #means_scores, conf_matrix, prec = model_evaluation_SFS(X_train, X_test, y_train, y_test, cricterior)

            #index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j]]))
            #index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)])) 

            if feature_vector[j][0] != []:
                index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][0]]))
                index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
                my_tree_gen = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
                accuracy_gen, df_gen = my_accuracy(test, my_tree_gen, name_classe)
                result_gen, means_scores_gen = compute_metrics_derive(df_gen, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)

            if feature_vector[j][1] != []:
                index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][1]]))
                index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
                my_tree_sel = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
                accuracy_sel, df_sel = my_accuracy(test, my_tree_sel, name_classe)
                result_sel, means_scores_sel = compute_metrics_derive(df_sel, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)

            #print("means_scores : ", means_scores)
            #print("my_tree : ", my_tree)
            #Use prepocessing data in X_train to transform the data before the training at the model 
            #model.fit(X_train, y_train)
            #scores = cross_val_score(model, X=X_train, y=y_train, cv=10, n_jobs=1)
            #means_scores = np.mean(scores)
            #print("my_tree :", my_tree)
            #print("max_scores : ", max_scores, "; Accuracy : ", accuracy, " ; means_scores : ", means_scores, "; feature_vector[j] : ", feature_vector[j], "; index_feature_not_selected: ", index_feature_not_selected)


            if type_cout in optimisation_minimisation:
                if means_scores_sel <= means_scores_gen:
                    means_scores = means_scores_sel
                    accuracy = accuracy_sel
                    my_tree = my_tree_sel
                    result = result_sel
                else:
                    means_scores = means_scores_gen
                    accuracy = accuracy_gen
                    my_tree = my_tree_gen
                    result = result_gen

                if means_scores < max_scores:#if means_scores < max_scores and accuracy > max_scores_acc:
                    #print("tchuente")
                    max_scores = means_scores
                    opt_tree = my_tree
                    max_scores_acc = accuracy
                    #opti_accuracy = accuracy
                    if type(my_tree) is dict:    
                        feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                    #print("feature_vector[j]& : ", feature_vector[j])
                    #print("feature_vector[j]-optimal: ", feature_vector[j])
                    opti_feature[i] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result]
                    index_opti_feature = i
                else:
                    if means_scores == max_scores:#if means_scores == max_scores:
                        if type(my_tree) is dict:    
                            feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                        ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                        feature_vector[itera] = deepcopy([deepcopy(feature_vector[j][0]), deepcopy(feature_vector[j][1])])
                        suite_index[itera] = i
                        itera = itera + 1
                    else:
                        #feature_vector[j][0].pop(len(feature_vector[j][0])-1)
                        feature_vector[j][1].pop(len(feature_vector[j][1])-1)
            elif type_cout in optimisation_maximisation:
                if means_scores_sel >= means_scores_gen:
                    means_scores = means_scores_sel
                    accuracy = accuracy_sel
                    my_tree = my_tree_sel
                    result = result_sel
                else:
                    means_scores = means_scores_gen
                    accuracy = accuracy_gen
                    my_tree = my_tree_gen
                    result = result_gen

                if means_scores > max_scores:#if means_scores > max_scores and accuracy > max_scores_acc:
                    #print("tchuente")
                    max_scores = means_scores
                    max_scores_acc = accuracy
                    opt_tree = my_tree
                    #opti_accuracy = accuracy
                    if type(my_tree) is dict:    
                        feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                    #####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                    ####print("feature_vector[j]& : ", feature_vector[j])
                    #print("feature_vector[j]-optimal: ", feature_vector[j])
                    opti_feature[i] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result]
                    index_opti_feature = i
                else:
                    if means_scores == max_scores:
                        ####feature_vector[j] = deepcopy(extract_feature_select_tree(my_tree))
                        if type(my_tree) is dict:    
                            feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
                        feature_vector[itera] = deepcopy([deepcopy(feature_vector[j][0]), deepcopy(feature_vector[j][1])])
                        suite_index[itera] = i
                        itera = itera + 1
                    else:
                        #feature_vector[j][0].pop(len(feature_vector[j][0])-1)
                        feature_vector[j][1].pop(len(feature_vector[j][1])-1)

            #print(opti_feature)
            if i >= beta-1:
                if j < len(feature_vector)-1:
                    j = j + 1
                    i = suite_index[j]                
                    if i >= beta-1:
                        divide = False
                else:
                    divide = False
            i = i + 1
        
        i = 0
        j = 0
        k = 0
        itera = 1
        divide = True
        
        if index_opti_feature != None:
            pred_feature_selected = deepcopy(opti_feature[index_opti_feature])
            sorted_vector = delete_element_list(pred_feature_selected[0], sorted_vector)
            if sorted_vector == []:
                 break
            feature_vector = dict()
            feature_vector[j] = deepcopy([deepcopy(pred_feature_selected[0]), deepcopy(pred_feature_selected[0])])
            opti_feature_boost[estimator_i] = deepcopy(pred_feature_selected)
            list_of_opti_feature.append(opti_feature)
        else:
            break
        
        print("sorted_vector : ", sorted_vector, "\n")
        print("Opti_feature : ", opti_feature, "\n")
        index_opti_feature = None
        beta = len(sorted_vector)
        opti_feature = dict()
        suite_index = dict()
        
        """if type_cout in optimisation_minimisation:
            max_scores = float('inf')
        elif type_cout in optimisation_maximisation:
            max_scores = -float('inf')
        """
    index_feature_not_selected = []
    my_tree = decision_tree_2(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
    accuracy, df = my_accuracy(test, my_tree, name_classe)
    results_all_data = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree), extract_feature_select_tree(my_tree)    
    return opti_feature, max_scores, sorted_vector, opt_tree, results_all_data, opti_feature_boost


In [12]:
def cross_validation_cs(X_frame, y_frame, decision_tree_cs, index_feature_not_selected, type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns_, random, cv):
    idx_montant = X_frame.columns.get_loc(name_montant)
    idx_taux = X_frame.columns.get_loc(name_taux)
    idx_duree = X_frame.columns.get_loc(name_duree)
    #print("y_frame : ", y_frame)
    X = X_frame.to_numpy()
    y_len = len(y_frame)
    y = y_frame.to_numpy().reshape((y_len, 1))
    if cv==1 :
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random)
        X_train = np.concatenate((X_train, y_train), axis=1)
        X_test = np.concatenate((X_test, y_test), axis=1)
        train = pd.DataFrame(data = X_train, columns = columns_)
        test = pd.DataFrame(data = X_test, columns = columns_)
        my_tree = decision_tree_cs(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
        accuracy, df = my_accuracy(test, my_tree, name_classe)
        result_, score = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
        return score, result_, accuracy
    else:
        scores = []
        accuracys = [] 
        precisions = [] 
        recalls = []
        accuracy_s = []
        loss_s = []
        kf = KFold(n_splits=cv, shuffle=True, random_state=random)
        for train_indices, test_indices in kf.split(X):
            X_train, y_train = X[train_indices], y[train_indices]
            X_test, y_test = X[test_indices], y[test_indices]
            #print("X_train :", X_train)
            #print("y_train :", y_train)
            X_train = np.concatenate((X_train, y_train), axis=1)
            X_test = np.concatenate((X_test, y_test), axis=1)
            X_train = pd.DataFrame(data = X_train, columns = columns_)
            X_test = pd.DataFrame(data = X_test, columns = columns_)
            #print("X_train' :", X_train)
            #print("y_train' :", y_train)
            my_tree = decision_tree_cs(X_train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            accuracy_, df = my_accuracy(X_test, my_tree, name_classe)
            result_, score = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            loss, accuracy, precision, recall = result_
            accuracy_s.append(accuracy_)
            accuracys.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            loss_s.append(loss)
            scores.append(score)
        #print("scores : ", scores, " , mean : ", mean(scores))    
        result_ = (mean(loss_s), mean(accuracys), mean(precisions), mean(recalls))    
        return mean(scores), result_, mean(accuracy_s)


def permute_weight(sorted_vector, feature_selected, features_dict):
    sub_feature = sorted_vector[:len(feature_selected)]
    p_vector = []
    for c in sub_feature:
        p_vector.append((c, features_dict[c]))
        
    s_vector = []
    for c in feature_selected:
        s_vector.append((c, features_dict[c]))
        
    p_vector = sorted(p_vector, key=lambda x:x[1][1], reverse=True)
    s_vector = sorted(s_vector, key=lambda x:x[1][1], reverse=True)

    
    for m in range(len(p_vector)):
        temp = sorted_vector[features_dict[p_vector[m][0]][0]]
        sorted_vector[features_dict[p_vector[m][0]][0]] = sorted_vector[features_dict[s_vector[m][0]][0]]
        sorted_vector[features_dict[s_vector[m][0]][0]] = temp
             
        temp = features_dict[p_vector[m][0]] 
        features_dict[p_vector[m][0]] = features_dict[s_vector[m][0]] 
        features_dict[s_vector[m][0]] = temp
        
    return sorted_vector, features_dict


def SBS_ARFS_cs(dataset, type_cout, alpha, set_feature, name_montant, name_taux, name_duree, name_classe, cricterior, decision_tree_cs, random, cv):
    #print("set_feature : ", set_feature)
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)
    
    dataset_y = dataset[name_classe]
    dataset_x = dataset.drop([name_classe], axis=1)
    
    columns = dataset.columns.tolist()
    len_columns_ = len(columns)
    
    feature = deepcopy(set_feature)
    results = dict()
    for i in range(len(feature)):
        diff_set = list(set(set_feature) - set([feature[i]]))

        index_feature_not_selected = list(set(range(len_columns_)) - set([dataset.columns.get_loc(col) for col in set_feature]))
        index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
        #my_tree = decision_tree_cs(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
        #accuracy, df_ = my_accuracy(test, my_tree, name_classe)
        #result, means_scores = compute_metrics_derive(df_, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
        means_scores_, result_, accuracy_ = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)
        results[i] = (means_scores_, diff_set, accuracy_, result_)
    #print(results)
    sorted_vector = sorted(results.items(), key=lambda x:x[1][0], reverse=False)
    #print(sorted_vector[0][1])
    return sorted_vector[0][1]    



def zyy_feature_selection_SFS_ARFSL_cs_update_(dataset, dfv, sep, type_cout, alpha, name_columns_drop, name_classe, name_montant,
                                                    name_taux, name_duree, cricterior, support_min=0.07, metric_v="confidence", threshold_min=0.4, random=10, sbs=False, decision_tree_cs="decision_tree", cv=3):
    start = time.time()
    divide = True
    d = 1
    i = d-1
    j = 0
    max_accuracy = float('inf')
    sorted_vector_ = processing_feature_selection_AR_base_Multi_Loss_(dataset.copy(), dfv, cricterior, alpha, sep, name_classe, name_montant, name_taux, name_duree, support_min, metric_v, threshold_min, random, decision_tree_cs)
    #sorted_vector = processing_feature_selection_AR_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #654sorted_vector = processing_feature_selection_AR_Inverse_oriented_classe(dataset, dfv, sep, name_classe, support_min, metric_v, threshold_min)
    #print(sorted_vector)
    #quit()
    #beta = len(sorted_vector)
    feature_dict = dict()
    sorted_vector = []
    for m in range(len(sorted_vector_)):
        sorted_vector.append(sorted_vector_[m][0])
        feature_dict[sorted_vector_[m][0]] = (m, sorted_vector_[m][1])
    #print(feature_dict)
    choix = 1
    list_vector = []
    actual_feature_selected = []
    check = False
    opti_feature = dict()
    suite_index = dict()
    k = 0
    feature_vector = dict()
    extract_information = dict()
    itera = 1
    feature_vector[j] = deepcopy([deepcopy([]), deepcopy([])])
    columns = dataset.columns.tolist()
    performance_prediction = []
        
    print("sorted_vector : ", sorted_vector)
    for m in columns:
        performance_prediction.append(-1)
    
    #X_train, X_test, y_train, y_test, columns_extract_all = preprocessing_dataset_(dataset.copy(), name_columns_drop, sep, name_classe, random)
    
    dataset_y = dataset[name_classe]
    dataset_x = dataset.drop(name_columns_drop, axis=1)
    
    #X_train, X_test, y_train, y_test = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=random)
    #train = pd.concat([X_train, y_train], axis=1)
    #test = pd.concat([X_test, y_test], axis=1)

    columns = dataset.columns.tolist()
    len_columns = len(columns)
    opt_tree = []
    
    idx_montant = dataset.columns.get_loc(name_montant)
    idx_taux = dataset.columns.get_loc(name_taux)
    idx_duree = dataset.columns.get_loc(name_duree)

    while divide == True:
        temp_i = -1
        if i not in opti_feature or sorted_vector[i]:    
            feature_vector[j][0] = sorted_vector[:i+1]
        elif sorted_vector[i] in opti_feature[i][0]:
            feature_vector[j][0] = sorted_vector[:i+1]
        else:
            feature_vector[j][0] = opti_feature[i][0]
            feature_vector[j][0].append(sorted_vector[i])
        if sorted_vector[i] not in feature_vector[j][1]:
            feature_vector[j][1].append(sorted_vector[i])

        #feature_vector[j][1].append(sorted_vector[i])
  
        print("i, j = ", i, ", ", j)
        #print("feature_vector : ", feature_vector)
        #data = dataset[np.concatenate((feature_vector[j], [name_classe]), axis=None)]
        
        #print("columns_extract_all : ", columns_extract_all)
            
        if feature_vector[j][0] != []:
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][0]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            #my_tree_gen = decision_tree_cs(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            #accuracy_gen, df_gen = my_accuracy(test, my_tree_gen, name_classe)
            #result_gen, means_scores_gen = compute_metrics_derive(df_gen, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            means_scores_gen, result_gen, accuracy_gen = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)

        
        if feature_vector[j][1] != []:
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in feature_vector[j][1]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            #my_tree_sel = decision_tree_cs(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            #accuracy_sel, df_sel = my_accuracy(test, my_tree_sel, name_classe)
            #result_sel, means_scores_sel = compute_metrics_derive(df_sel, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            means_scores_sel, result_sel, accuracy_sel = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)
           
        
        print("feature_vector[j] : ", feature_vector[j][1], " sorted_vector[i] : ", sorted_vector[i], " means_scores_sel : ", means_scores_sel)
        ###addd
        
        if len(feature_vector[j][0]) not in opti_feature:
            performance_prediction[len(feature_vector[j][0])-1] = means_scores_gen
            opti_feature[len(feature_vector[j][0])] = [deepcopy(feature_vector[j][0]), means_scores_gen, len(feature_vector[j][0]), accuracy_gen, result_gen]
            #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(feature_vector[j][0]), feature_dict.copy())
            #temp_i = len(feature_vector[j][0]) - 1
        else:
            if means_scores_gen < opti_feature[len(feature_vector[j][0])][1] and len(feature_vector[j][0])-1 in opti_feature and means_scores_gen <= opti_feature[len(feature_vector[j][0])-1][1]:
                performance_prediction[len(feature_vector[j][0])-1] = means_scores_gen
                opti_feature[len(feature_vector[j][0])] = [deepcopy(feature_vector[j][0]), means_scores_gen, len(feature_vector[j][0]), accuracy_gen, result_gen]
                #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(feature_vector[j][0]), feature_dict.copy())
                #temp_i = len(feature_vector[j][0]) - 1
            else:
                if means_scores_gen < opti_feature[len(feature_vector[j][0])][1]:
                    performance_prediction[len(feature_vector[j][0])-1] = means_scores_gen
                    opti_feature[len(feature_vector[j][0])] = [deepcopy(feature_vector[j][0]), means_scores_gen, len(feature_vector[j][0]), accuracy_gen, result_gen]
                    #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(feature_vector[j][0]), feature_dict.copy())
                    #temp_i = len(feature_vector[j][0]) - 1
                pass
        ########
                        
        if means_scores_sel < means_scores_gen:
            means_scores = means_scores_sel
            accuracy = accuracy_sel
            #my_tree = my_tree_sel
            result = result_sel
            feature_selected = deepcopy(feature_vector[j][1])
        else:
            if means_scores_sel < means_scores_gen:
                means_scores = means_scores_gen
                accuracy = accuracy_gen
                #my_tree = my_tree_gen
                result = result_gen
                feature_selected = deepcopy(feature_vector[j][0])
            else:
                if len(feature_vector[j][1]) >= len(feature_vector[j][0]):
                    means_scores = means_scores_gen
                    accuracy = accuracy_gen
                    #my_tree = my_tree_gen
                    result = result_gen
                    feature_selected = deepcopy(feature_vector[j][0])
                else:
                    means_scores = means_scores_sel
                    accuracy = accuracy_sel
                    #my_tree = my_tree_sel
                    result = result_sel
                    feature_selected = deepcopy(feature_vector[j][1])

        
        if means_scores < max_accuracy:
            max_accuracy = means_scores
            #opt_tree = my_tree
            max_scores_acc = accuracy
            z = len(feature_selected)
            performance_prediction[z-1] = means_scores
            #if type(my_tree) is dict:    
            #    feature_vector[j][1] = deepcopy(extract_feature_select_tree(my_tree))
            opti_feature[z] = [deepcopy(feature_vector[j][1]), means_scores, len(feature_vector[j][1]), accuracy, result]       
            feature_vector[j][1] = deepcopy(feature_selected)
            sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(feature_selected), feature_dict.copy())
            temp_i = z - 1
            print("best value of j = ", j)
        else:
            feature_vector[j][1].pop(len(feature_vector[j][1])-1)
        
        
        if sbs:
            for q in range(i):
                if q+1 in opti_feature:
                    set_feature_use = deepcopy(opti_feature[q+1][0])                
                    check == True
                    while(check == True):
                        if len(set_feature_use) > 1:
                            scores, set_feature, accuracy_, result = SBS_ARFS_cs(dataset, type_cout, alpha, deepcopy(set_feature_use), name_montant, name_taux, name_duree, name_classe, cricterior, decision_tree_cs, random, cv)
                            
                            #scores, set_feature, accuracy_, result, my_tree = SBS_ARFS_cs(dataset_x, dataset_y, train, test, type_cout, alpha, deepcopy(set_feature_use), name_montant, name_taux, name_duree, name_classe, cricterior, decision_tree_cs)
                            if len(set_feature) not in opti_feature:
                                performance_prediction[len(set_feature)-1] = scores
                                opti_feature[len(set_feature)] = [deepcopy(set_feature), scores, len(set_feature), accuracy_, result]
                                #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(set_feature), feature_dict.copy())
                                #temp_i = len(set_feature) - 1
                            elif scores < opti_feature[len(set_feature)][1] and len(set_feature)-1 in opti_feature and scores <= opti_feature[len(set_feature)-1][1]:
                                performance_prediction[len(set_feature)-1] = scores
                                opti_feature[len(set_feature)] = [deepcopy(set_feature), scores, len(set_feature), accuracy_, result]
                                #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(set_feature), feature_dict.copy())
                                #temp_i = len(set_feature) - 1
                            elif scores < opti_feature[len(set_feature)][1]:
                                performance_prediction[len(set_feature)-1] = scores
                                opti_feature[len(set_feature)] = [deepcopy(set_feature), scores, len(set_feature), accuracy_, result]
                                #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(set_feature), feature_dict.copy())
                                #temp_i = len(set_feature) - 1
                            else:
                                check = False

                            if scores < max_accuracy:
                                feature_deleted = list(set(set_feature_use) - set(set_feature))
                                feature_deleted = feature_deleted[0]

                                max_accuracy = scores
                                #opt_tree = my_tree
                                max_scores_acc = accuracy_

                                feature_vector[j][0] = deepcopy(list(set(feature_vector[j][0]) - set([feature_deleted])))
                                feature_vector[j][1] = deepcopy(set_feature)
                                actual_feature_selected = deepcopy(set_feature)
                                set_feature_use = deepcopy(set_feature)
                                z = len(set_feature)
                                opti_feature[z] = [deepcopy(set_feature), scores, z, accuracy_, result]
                                sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(set_feature), feature_dict.copy())
                                temp_i = z-1
                                performance_prediction[z-1] = scores
                        else:
                            check = False

        for q in range(i+1):
            if q not in opti_feature:
                test_feature = []
            elif q in opti_feature:
                test_feature = deepcopy(opti_feature[q][0])
            if sorted_vector[i] not in test_feature:
                test_feature.append(sorted_vector[i])
            elif sorted_vector[i] in test_feature:
                test_feature = []

            if test_feature != []:
                index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in test_feature]))
                index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
                #my_tree_test = decision_tree_cs(train, deepcopy(index_feature_not_selected), idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
                #accuracy_test, df_test = my_accuracy(test, my_tree_test, name_classe)
                #result_test, means_scores_test = compute_metrics_derive(df_test, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
                means_scores_test, result_test, accuracy_test = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)

                
                if len(test_feature)-1 in opti_feature and len(test_feature) in opti_feature and means_scores_test < opti_feature[len(test_feature)-1][1] and means_scores_test < opti_feature[len(test_feature)][1]:
                    performance_prediction[len(test_feature)-1] = means_scores_test
                    opti_feature[len(test_feature)] = [deepcopy(test_feature), means_scores_test, len(test_feature), accuracy_test, result_test]
                    #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(test_feature), feature_dict.copy())
                    #temp_i = len(test_feature) - 1
                elif len(test_feature) in opti_feature and means_scores_test < opti_feature[len(test_feature)][1]:
                    performance_prediction[len(test_feature)-1] = means_scores_test
                    opti_feature[len(test_feature)] = [deepcopy(test_feature), means_scores_test, len(test_feature), accuracy_test, result_test]
                    #sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(test_feature), feature_dict.copy())
                    #temp_i = len(test_feature) - 1

                if means_scores_test < max_accuracy:
                    max_accuracy = means_scores_test
                    
                    #opt_tree = my_tree_test
                    max_scores_acc = accuracy_test
                    
                    actual_feature_selected = deepcopy(test_feature) 
                    feature_vector[j][1] = deepcopy(test_feature)
                    z = len(test_feature)
                    performance_prediction[z-1] = means_scores_test
                    opti_feature[z] = [deepcopy(test_feature), means_scores_test, z, accuracy_test, result_test]
                    
                    sorted_vector, feature_dict = permute_weight(deepcopy(sorted_vector), deepcopy(test_feature), feature_dict.copy())
                    temp_i = z-1
                                
        
        if temp_i != -1:
            i = temp_i
        #print(opti_feature)
        if i >= len(sorted_vector)-1:
            if j < len(feature_vector)-1:
                sorted_vector = list_vector[j]
                j = j + 1
                i = suite_index[j]                
                if i >= len(sorted_vector)-1:
                    divide = False
            else:
                divide = False
        i = i + d

    for j in range(len(sorted_vector)):
        if sorted_vector[:j] != []:
            #X_train_selected_gen, X_test_selected_gen, columns_extract_gen = selected_dataset_(columns_extract_all, deepcopy(sorted_vector[:j]), sep, X_train.copy(), X_test.copy())
            #means_scores_gen, conf_matrix_gen, prec_gen, feature_selected_gen = model_evaluation_SFS_(X_train_selected_gen.copy(), X_test_selected_gen.copy(), y_train.copy(), y_test.copy(), columns_extract_gen, columns, sep, cricterior)
                        
            index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in sorted_vector[:j]]))
            index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
            #my_tree = decision_tree_cs(train, deepcopy(index_feature_not_selected), idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
            #accuracy, df = my_accuracy(test, my_tree, name_classe)
            #result, means_scores = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
            means_scores, result, accuracy = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)
            
            if len(sorted_vector[:j]) in opti_feature and means_scores < opti_feature[len(sorted_vector[:j])][1]:
                performance_prediction[len(sorted_vector[:j])-1] = means_scores
                opti_feature[len(sorted_vector[:j])] = [deepcopy(sorted_vector[:j]), means_scores, len(sorted_vector[:j]), accuracy, result]
                if means_scores < max_accuracy:
                    max_accuracy = means_scores
                    #opt_tree = my_tree
                    max_scores_acc = accuracy
                    actual_feature_selected = deepcopy(len(sorted_vector[:j]))

            if j in opti_feature:
                if sorted_vector[j] not in opti_feature[j][0]: 
                    second_feature = deepcopy(opti_feature[j][0])
                    second_feature.append(sorted_vector[j])

                    #X_train_selected_gen, X_test_selected_gen, columns_extract_gen = selected_dataset_(columns_extract_all, deepcopy(second_feature), sep, X_train.copy(), X_test.copy())
                    #means_scores_gen, conf_matrix_gen, prec_gen, feature_selected_gen = model_evaluation_SFS_(X_train_selected_gen.copy(), X_test_selected_gen.copy(), y_train.copy(), y_test.copy(), columns_extract_gen, columns, sep, cricterior)
                    
                    
                    index_feature_not_selected = list(set(range(len_columns)) - set([dataset.columns.get_loc(col) for col in second_feature]))
                    index_feature_not_selected = list(set(index_feature_not_selected) - set([dataset.columns.get_loc(name_classe)]))
                    #my_tree = decision_tree_cs(train, deepcopy(index_feature_not_selected), idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
                    #accuracy, df = my_accuracy(test, my_tree, name_classe)
                    #result, means_scores = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree)
                    means_scores, result, accuracy = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)
                    
                    
                    if len(second_feature)-1 in opti_feature and means_scores < opti_feature[len(second_feature)-1][1] and len(second_feature) in opti_feature and means_scores < opti_feature[len(second_feature)][1]:
                        performance_prediction[len(second_feature)-1] = means_scores
                        opti_feature[len(second_feature)] = [deepcopy(second_feature), means_scores, len(second_feature), accuracy, result]
                    elif len(second_feature) in opti_feature and means_scores < opti_feature[len(second_feature)][1]:
                        performance_prediction[len(second_feature)-1] = means_scores
                        opti_feature[len(second_feature)] = [deepcopy(second_feature), means_scores, len(second_feature), accuracy, result]
                        
                    if means_scores < max_accuracy:
                        max_accuracy = means_scores
                        #opt_tree = my_tree
                        max_scores_acc = accuracy
                        actual_feature_selected = deepcopy(second_feature) 


    print("performance_prediction : ", performance_prediction)
    end = time.time()
    start_ = time.time()
    index_feature_not_selected = []
    #my_tree = decision_tree_cs(train, index_feature_not_selected, idx_montant, idx_taux, idx_duree, min_samples=5, max_depth=5, metric=cricterior)
    #accuracy, df = my_accuracy(test, my_tree, name_classe)
    #results_all_data = compute_metrics_derive(df, type_cout, alpha, name_classe, name_montant, name_taux, name_duree), extract_feature_select_tree(my_tree)    
    means_scores, result, accuracy = cross_validation_cs(dataset_x, dataset_y, decision_tree_cs, deepcopy(index_feature_not_selected), type_cout, alpha, name_classe, name_montant, name_taux, name_duree, cricterior, columns, random, cv)
    end_ = time.time() 
    return opti_feature, max_accuracy, sorted_vector, (means_scores, result)
    


In [24]:
datas = pd.read_csv('../../dataset/german.data',sep=' ')
datas = datas.dropna().reset_index(drop=True)
name_classe = "A20"
datas[name_classe] = datas[name_classe].replace([2, 1], [0, 1])


datas

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A11,A12,A13,A14,A15,A16,A17,A18,A19,A20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,0
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,0


In [25]:
sep = "__"

columns = datas.columns

dfv = pd.DataFrame()
for col in columns:
    dfr = pd.get_dummies(datas[col], prefix_sep=sep, prefix = col)
    dfv = pd.concat([dfv, dfr], axis=1)

dfv

,A0__A11,A0__A12,A0__A13,A0__A14,A1__4,A1__5,A1__6,A1__7,A1__8,A1__9,...,A16__A173,A16__A174,A17__1,A17__2,A18__A191,A18__A192,A19__A201,A19__A202,A20__0,A20__1
0,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,1,1,0,0,1
1,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,1,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,1,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,0,1
996,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,1,0,0,1
997,0,0,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,1
998,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,1,0


In [26]:
# feature_selection_SFS_ARFS_Sensitive_Multi_Loss



#name_classe = "ENIMPAYEOUPAS"
name_classe = "A20"
#name_classe = "A15"


name_columns_drop = [name_classe]
support_min = 0.1
metric_v = "confidence"
threshold_min = 0.1
type_cout = "pessimiste"
cricterior = "pessimiste"
alpha = 0.0 
global ALPHA_2
ALPHA_2 = 0.0
random = 3
sbs=True
decision_tree_cs = decision_tree_2
cv = 5

#name_montant = 'Montant'
#name_taux = 'Taux'
#name_duree = 'Nbreech.'

name_montant = 'A4'
name_taux = 'A7'
name_duree = 'A1'

#name_montant = 'A14'
#name_taux = 'A3'
#name_duree = 'A5'

opti_feature, max_accuracy, sorted_vector, results_all_data = zyy_feature_selection_SFS_ARFSL_cs_update_(datas.copy(), dfv, sep, type_cout, alpha, name_columns_drop, name_classe, 
                                                name_montant, name_taux, name_duree, cricterior, support_min, metric_v, threshold_min, random, sbs, decision_tree_cs, cv)

print("Optimum feature : ", opti_feature) 
print("\n\n")
print("Sorted vector : ", sorted_vector)
print("\n\n")
print("Minimal cost : ", max_accuracy)
print("\n\n")
print("Results all data : ", results_all_data)


sorted_vector :  [('A1', 17242.351304347827), ('A3', 25979.047023809522), ('A11', 121594.75301204818), ('A6', 125328.28318584069), ('A7', 157316.80462184874), ('A10', 157664.9491525424), ('A8', 175911.08394160582), ('A14', 179229.65217391303), ('A18', 187573.28712871287), ('A16', 200033.55238095237), ('A2', 202921.16981132075), ('A0', 227054.72627737228), ('A15', 237084.99210110586), ('A13', 244179.5380835381), ('A5', 306551.5456053068), ('A17', 308310.6106508876), ('A9', 317318.1389195149), ('A19', 353737.5202492212)]
set_feature :  [('A4', 41187.662500000006), ('A12', 45753.0225)]
sorted_vector :  ['A1', 'A3', 'A11', 'A6', 'A7', 'A10', 'A8', 'A14', 'A18', 'A16', 'A2', 'A0', 'A15', 'A13', 'A5', 'A17', 'A9', 'A19', 'A4', 'A12']
i, j =  0 ,  0
feature_vector[j] :  ['A1']  sorted_vector[i] :  A1  means_scores_sel :  26341.804000000004
best value of j =  0
i, j =  1 ,  0
feature_vector[j] :  ['A1', 'A3']  sorted_vector[i] :  A3  means_scores_sel :  34298.26466666667
i, j =  1 ,  0
feature

In [ ]:
23042.7640

In [ ]:
2, 2